# DATA PREPARATION AND METHODS LSTM

In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Wczytanie danych z kotła
# List of file paths
rel_path = '..\\dane\\sodowy_no_missing_values\\'
file_paths = ['part_326.csv', 'part_368.csv', 'part_369.csv',
              'part_821.csv', 'part_822.csv', 'part_942.csv',
              'part_2757.csv', 'part_2776.csv', 'part_2783.csv',
              'part_2789.csv', 'part_2793.csv', 'part_2878.csv',
              'part_2883.csv', 'part_3143.csv', 'part_3188.csv',
              'part_3588.csv', 'part_3660.csv', 'part_4381.csv',
              'part_4425.csv', 'part_6850.csv']
file_paths = [rel_path + file for file in file_paths]


In [ ]:
#File paths for each type of cross-val dataset
train_file_paths = file_paths[:15]
val_file_paths = file_paths[15:16]
test_file_paths = file_paths[16:]

In [ ]:
#Funkcja odpowiedzialna za normalizację danych na podstawie podanych zakresów
def norm(data):
  data_norm = data.copy()
  for column in data_norm:
    if(ranges[column] != (0, 0)): #jeśli jest podany zakres
      data_norm[[column]] = (data_norm[[column]]-ranges[column][0])/(ranges[column][1]-ranges[column][0])
  return data_norm

In [ ]:
# Wczytujemy dane z plików i sklejamy
data = pd.concat([pd.read_csv(file, delimiter=';') for file in file_paths], ignore_index=True)

In [ ]:
#Słownik z zakresami dla danych z każdej kolumuny
ranges = {column:(min(data[column]),max(data[column])) for column in data.columns}
ranges['time'] = (0,0) #Czasu nie normalizujemy

In [ ]:
#Połączone zbiory znormalizowanych danych treningowych, walidacyjnych i testowych bez wierszy wyznaczających granicę miedzy kolejnymi plikami
train_file_data = pd.concat([norm(pd.read_csv(file, delimiter=';')).iloc[2:].reset_index(drop = True) for file in train_file_paths], ignore_index=True)
val_file_data = pd.concat([norm(pd.read_csv(file, delimiter=';')).iloc[2:].reset_index(drop = True) for file in val_file_paths], ignore_index=True)
test_file_data = pd.concat([norm(pd.read_csv(file, delimiter=';')).iloc[2:].reset_index(drop = True) for file in test_file_paths], ignore_index=True)

In [ ]:
#Resampling (taking every 10th row)

#Testing data
test_file_data_resampled = test_file_data.iloc[::10]
test_file_data_resampled.reset_index(inplace= True, drop= True)

#Validation data
val_file_data_resampled = val_file_data.iloc[::10]
val_file_data_resampled.reset_index(inplace= True, drop= True)

#Training data
train_file_data_resampled = train_file_data.iloc[::10]
train_file_data_resampled.reset_index(inplace= True, drop= True)

In [ ]:
val_file_data

In [ ]:
val_file_data.shape, test_file_data.shape

In [ ]:
ranges

In [ ]:
test_file_data_resampled

In [ ]:
plt.plot(data['v_16001_M01_ACT_RPM_OUT'])

In [ ]:
data_norm = norm(data)

In [ ]:
data_norm

In [ ]:
#Funkcja odpowiedzialna za denormalizację wyników
def denorm(data_norm, pred_name):
  data_denorm = data_norm.copy()
  for column in data_denorm:
    if(ranges[pred_name] != (0, 0)): #jeśli jest podany zakres
      data_denorm[[column]] = (data_denorm[[column]]*(ranges[pred_name][1]-ranges[pred_name][0]))+ranges[pred_name][0]
  return data_denorm

In [ ]:
#Wyodrębnienie zbiorów danych niezbędnych do stworzenia każdego z modeli (jako listy nazw kolumn)

#Model obrotów wentylatora WS1
#v_16001_M01_ACT_RPM_OUT
col_WS1 = ['v_16001_M01_ACT_RPM_OUT', 'v_11502_PIZ', 'v_11503_TI', 'v_16524_PI','v_16532_TI', 'v_16542_PIZ', 'v_16627_TI','v_17516_FIC','v_17546_FICZ','v_19587_FIZ']

#Model obrotów wentylatora WS2
#v_16002_M01_ACT_RPM_OUT
col_WS2 = ['v_16002_M01_ACT_RPM_OUT', 'v_16542_PIZ', 'v_16563_PIZ', 'v_22684_FI', 'v_17516_FIC', 'v_16513_PI', 'v_16512_PI', 'v_17546_FICZ', 'v_11502_PIZ', 'v_16627_TI', 'v_16534_TI' ]

#Model pomiaru ciśnienia KP1 w komorze paleniskowej
#v_16505_PI	Ciśnienie na wyjsciu KP 1
col_KP1 = ['v_16505_PI', 'v_16502_PIZ', 'v_16512_PI', 'v_16509_PIC_PIDA_SP', 'v_19603_FI']

#Model pomiaru ciśnienia K01 na wyjściu z kotła
#v_16512_PI	Ciśnienie w środku KO 1
col_KO1 = ['v_16512_PI', 'v_16525_PI', 'v_22684_FI', 'v_17516_FIC','v_16627_TI', 'v_11502_PIZ', 'v_16510_TI', 'v_16511_TI', 'v_16533_TI', 'v_17546_FICZ', 'v_16532_TI', 'v_16629_TI', 'v_17501_FICZ', 'v_16528_AIZ', 'v_22559_TI', 'v_16529_AIZ', 'v_15554_FICZ', 'v_16505_PI', 'v_16502_PIZ']

#Model pomiaru ciśnienia K02 na wyjściu z kotła
#v_16513_PI	Ciśnienie w środku KO 2
col_KO2 = ['v_16513_PI', 'v_16525_PI', 'v_22684_FI','v_16627_TI', 'v_11502_PIZ', 'v_16510_TI', 'v_16511_TI', 'v_16533_TI', 'v_17546_FICZ', 'v_16532_TI', 'v_17516_FIC', 'v_16629_TI', 'v_17501_FICZ', 'v_16528_AIZ', 'v_22559_TI', 'v_16529_AIZ', 'v_15554_FICZ', 'v_16502_PIZ']

#Model pomiaru ciśnienia spalin KS1
#v_16524_PI	Podciśnienie na wejściu KS 1
col_KS1 = ['v_16524_PI', 'v_16563_PIZ', 'v_16542_PIZ', 'v_17516_FIC', 'v_16512_PI']

#Model pomiaru ciśnienia spalin KS2
#v_16525_PI	Podciśnienie na wejściu KS 2
col_KS2 = ['v_16525_PI', 'v_16563_PIZ', 'v_16542_PIZ', 'v_17516_FIC', 'v_16512_PI', 'v_16513_PI']

#Model pomiaru zawartości tlenu w spalinach O1
#v_16528_AIZ	Tlen w spalinach w kotle pom 1
col_O1 = ['v_16528_AIZ', 'v_19587_FIZ', 'v_22560_FIC', 'v_22684_FI', 'v_19588_TIZ', 'v_16511_TI', 'v_16510_TI', 'v_16509_PIC_PIDA_OP', 'v_16524_PI', 'v_16512_PI', 'v_16513_PI', 'v_19518_TIC', 'v_16627_TI', 'v_17516_FIC', 'v_15549_FICZ', 'v_19517_TIC', 'v_15557_FICZ']

#Model pomiaru zawartości tlenu w spalinach O2
#v_16529_AIZ	Tlen w spalinach w kotle pom 2
col_O2 = ['v_16529_AIZ', 'v_19587_FIZ', 'v_22560_FIC', 'v_22684_FI', 'v_16510_TI', 'v_19588_TIZ', 'v_16509_PIC_PIDA_OP']

#Model pomiaru zawartości tlenu w spalinach O3
#v_16530_AIZ  Tlen w spalinach w kotle pom 3
col_O3 = ['v_16530_AIZ', 'v_19587_FIZ', 'v_22560_FIC', 'v_22684_FI', 'v_16510_TI', 'v_19517_TIC', 'v_19588_TIZ', 'v_15549_FICZ', 'v_16509_PIC_PIDA_OP', 'v_16525_PI', 'v_16512_PI', 'v_16511_TI', 'v_15557_FICZ']


In [ ]:
#Funkcja przetwarzająca szereg czasowy na próbki w postaci: window_size wartości poprzednich -> wartość następna
def data_to_X_y(data, window_size=5):
    data_as_np = data.to_numpy()
    X = []
    y = []
    for i in range(len(data_as_np)-window_size):
        #Jako wejście bierzemy ciąg o długości WINDOW_SIZE-1
        row = [[a] for a in data_as_np[i:i+window_size]]
        X.append(row)
        #Jako etykietę bierzemy następną wartość po ciągu
        label = data_as_np[i+window_size]
        y.append(label)
    return np.array(X), np.array(y)


In [ ]:
#Funkcja odpowiedzialna za podział danych na zbiór treningowy, walidacyjny oraz testowy (przetwarzane są szeregi czasowe, więc nie mieszamy danych) 
##############
#  W TYM PRZYPADKU MAMY JUŻ DANE PODZIELONE NA PLIKI, WIĘC TA FUNKCJA NIE BĘDZIE WYKORZYSTYWANA
##############
def data_to_cross_val(sequences, variables, labels):
  #Pobieramy rozmiar zbioru danych wejściowych
  size = sequences.shape[0]
  train = int(size*0.8)
  val = int(size*0.9)
  
  sequences_train, variables_train, labels_train = sequences[:train], variables[:train], labels[:train]
  sequences_val, variables_val, labels_val = sequences[train:val], variables[train:val], labels[train:val]
  sequences_test, variables_test, labels_test = sequences[val:], variables[val:], labels[val:]
  
  return sequences_train, variables_train, labels_train, sequences_val, variables_val, labels_val, sequences_test, variables_test, labels_test
  

In [ ]:
#Keras imports
from keras.models import Sequential, Model
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam
from keras.models import load_model

In [ ]:
#Funkcja odpowiedzialna za tworzenie modelu
def create_model(var_num):
    # Define input layers
    sequence_input = Input(shape=(None, 1))  #Values for the past time steps (input can have any window size)
    variables_input = Input(shape=(var_num,))  # Variables for the current time step

    #LSTM layers with dropout for sequences
    lstm_sequence = LSTM(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(sequence_input)
    lstm_sequence = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(lstm_sequence)

    #Dense layer for additional variables at current time step
    dense_variables = Dense(32, activation='relu')(variables_input)
    dense_variables = Dropout(0.2)(dense_variables)
    
    # Concatenate the outputs of both layers
    merged = concatenate([lstm_sequence, dense_variables])
    
    # Dense layer for regression with linear activation function
    output = Dense(1, activation='linear', kernel_regularizer='l2')(merged)

    # Create the model
    model = Model(inputs=[sequence_input, variables_input], outputs=output)
    return model

In [ ]:
#Funkcja odpowiedzialna za przewidywanie kolejnych wartości -> wersja ze średnią ważoną
def predict(model, input_sequence, input_variables, window_size, num_steps, labels):
    predictions = np.zeros(num_steps) #empty numpy array for storing predictions
    sequence_input = np.array(input_sequence[0]) #We take only the first window_size values from the data
    variables_input = np.array(input_variables) #We create a temporary variable for additional variables data
    
    for i in range(num_steps):
        
        #The data needs to be reshaped before passing into the model so that it fits earlier defined input sizes
        next_value = model.predict([sequence_input.reshape(1, window_size, 1), variables_input[i].reshape(1,variables_input.shape[1])], verbose=0) 
        
        #Adding predicted values to predictions array
        predictions[i] = next_value[0][0]
        
        # Update the input sequence for the next iteration
        sequence_input = np.roll(sequence_input, -1)  # Shift left one place
        sequence_input[-1] = ((9 * next_value[0][0] + labels[i]) / 10)  # Insert the new value (weighted average) at the end
        
        #Counting how many steps forward have already been predicted and printing it to the console
        if i % 500 == 0:
            #os.system('cls')
            print(i, '/', num_steps)
    print("Done")
    return predictions


In [ ]:
# #Funkcja odpowiedzialna za przewidywanie kolejnych wartości -> wersja bez średniej ważonej
# def predict(model, input_sequence, input_variables, window_size, num_steps):
#     predictions = np.zeros(num_steps) #empty numpy array for storing predictions
#     sequence_input = np.array(input_sequence[0]) #We take only the first window_size values from the data
#     variables_input = np.array(input_variables) #We create a temporary variable for additional variables data
    
#     for i in range(num_steps):
        
#         #The data needs to be reshaped before passing into the model so that it fits earlier defined input sizes
#         next_value = model.predict([sequence_input.reshape(1, window_size, 1), variables_input[i].reshape(1,variables_input.shape[1])], verbose=0) 
        
#         #Adding predicted values to predictions array
#         predictions[i] = next_value[0][0]
        
#         # Update the input sequence for the next iteration
#         sequence_input = np.roll(sequence_input, -1)  # Shift left one place
#         sequence_input[-1] = next_value[0][0]  # Insert the new value at the end
        
#         #Counting how many steps forward have already been predicted and printing it to the console
#         if i % 500 == 0:
#             #os.system('cls')
#             print(i, '/', num_steps)
#     print("Done")
#     return predictions


In [ ]:
#Funkcja odpowedzialna za wyświetlanie wyników
def display(train_results, val_results, test_results, which_dataset, plot_name):
    if which_dataset=='all':
        # Create a figure and subplots for each dataset from cross validation
        fig, axs = plt.subplots(
            3, 2,               # 3 rows, 2 columns
            figsize=(20, 16),    # Width = 10 , height = 8 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0, 0].plot(train_results['Train Predictions'], label='Model')
        axs[0, 0].plot(train_results['Actuals'], label='Real')
        axs[0, 0].set_title('Training dataset')
        axs[0, 0].set_xlabel('Time')
        axs[0, 0].set_ylabel('Value')
        axs[0, 0].legend()

        axs[1, 0].plot(val_results['Val Predictions'], label='Model')
        axs[1, 0].plot(val_results['Actuals'], label='Real')
        axs[1, 0].set_title('Validation dataset')
        axs[1, 0].set_xlabel('Time')
        axs[1, 0].set_ylabel('Value')
        axs[1, 0].legend()

        axs[2, 0].plot(test_results['Test Predictions'], label='Model')
        axs[2, 0].plot(test_results['Actuals'], label='Real')
        axs[2, 0].set_title('Testing dataset')
        axs[2, 0].set_xlabel('Time')
        axs[2, 0].set_ylabel('Value')
        axs[2, 0].legend()

        # Plot difference between model and real data
        axs[0, 1].plot(train_results['Train Predictions'] - train_results['Actuals'], label='Difference', color='red')
        axs[0, 1].set_title('Training dataset (Difference)')
        axs[0, 1].set_xlabel('Time')
        axs[0, 1].set_ylabel('Value')
        axs[0, 1].legend()

        axs[1, 1].plot(val_results['Val Predictions']- val_results['Actuals'], label='Difference', color='red' )
        axs[1, 1].set_title('Validation dataset (Difference)')
        axs[1, 1].set_xlabel('Time')
        axs[1, 1].set_ylabel('Value')
        axs[1, 1].legend()

        axs[2, 1].plot(test_results['Test Predictions'] - test_results['Actuals'], label='Difference', color='red')
        axs[2, 1].set_title('Testing dataset (Difference)')
        axs[2, 1].set_xlabel('Time')
        axs[2, 1].set_ylabel('Value')
        axs[2, 1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()
        
    elif which_dataset=='train':
        # Create a figure and subplots for testing dataset
        fig, axs = plt.subplots(
            2, 1,               # 2 rows, 1 column
            figsize=(20, 5),    # Width = 20 , height = 5 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0].plot(train_results['Train Predictions'], label='Model')
        axs[0].plot(train_results['Actuals'], label='Real')
        axs[0].set_title('Training dataset')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Value')
        axs[0].legend()

        # Plot difference between model and real data
        axs[1].plot(train_results['Train Predictions'] - train_results['Actuals'], label='Difference', color='red')
        axs[1].set_title('Training dataset (Difference)')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel('Value')
        axs[1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()

    elif which_dataset=='val':
        # Create a figure and subplots for testing dataset
        fig, axs = plt.subplots(
            2, 1,               # 2 rows, 1 column
            figsize=(20, 5),    # Width = 20 , height = 5 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0].plot(val_results['Val Predictions'], label='Model')
        axs[0].plot(val_results['Actuals'], label='Real')
        axs[0].set_title('Validation dataset')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Value')
        axs[0].legend()

        # Plot difference between model and real data
        axs[1].plot(val_results['Val Predictions'] - val_results['Actuals'], label='Difference', color='red')
        axs[1].set_title('Validation dataset (Difference)')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel('Value')
        axs[1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()
    
    elif which_dataset=='test':
        # Create a figure and subplots for testing dataset
        fig, axs = plt.subplots(
            2, 1,               # 2 rows, 1 column
            figsize=(20, 5),    # Width = 20 , height = 5 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0].plot(test_results['Test Predictions'], label='Model')
        axs[0].plot(test_results['Actuals'], label='Real')
        axs[0].set_title('Testing dataset')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Value')
        axs[0].legend()

        # Plot difference between model and real data
        axs[1].plot(test_results['Test Predictions'] - test_results['Actuals'], label='Difference', color='red')
        axs[1].set_title('Testing dataset (Difference)')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel('Value')
        axs[1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()

    else: print("Please enter a valid dataset to display!")

In [ ]:
#Funkcja odpowiedzialna za wyświetlanie wyników z podanego przedziału
def display_window(train_results, val_results, test_results, which_dataset, plot_name, train=0, val=0, test=0, window_train=3000, window_val=3000, window_test=3000):
    if which_dataset=='all':
        # Create a figure and subplots for each dataset from cross validation
        fig, axs = plt.subplots(
            3, 2,               # 3 rows, 2 columns
            figsize=(20, 16),    # Width = 10 , height = 8 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0, 0].plot(train_results['Train Predictions'][train:train+window_train], label='Model') #
        axs[0, 0].plot(train_results['Actuals'][train:train+window_train], label='Real')
        axs[0, 0].set_title('Training dataset')
        axs[0, 0].set_xlabel('Time')
        axs[0, 0].set_ylabel('Value')
        axs[0, 0].legend()

        axs[1, 0].plot(val_results['Val Predictions'][val:val+window_val], label='Model') #
        axs[1, 0].plot(val_results['Actuals'][val:val+window_val], label='Real')
        axs[1, 0].set_title('Validation dataset')
        axs[1, 0].set_xlabel('Time')
        axs[1, 0].set_ylabel('Value')
        axs[1, 0].legend()

        axs[2, 0].plot(test_results['Test Predictions'][test:test+window_test], label='Model') #
        axs[2, 0].plot(test_results['Actuals'][test:test+window_test], label='Real')
        axs[2, 0].set_title('Testing dataset')
        axs[2, 0].set_xlabel('Time')
        axs[2, 0].set_ylabel('Value')
        axs[2, 0].legend()

        # Plot difference between model and real data
        axs[0, 1].plot(train_results['Train Predictions'][train:train+window_train] - train_results['Actuals'][train:train+window_train], label='Difference', color='red') #
        axs[0, 1].set_title('Training dataset (Difference)')
        axs[0, 1].set_xlabel('Time')
        axs[0, 1].set_ylabel('Value')
        axs[0, 1].legend()

        axs[1, 1].plot(val_results['Val Predictions'][val:val+window_val]- val_results['Actuals'][val:val+window_val], label='Difference', color='red' ) #
        axs[1, 1].set_title('Validation dataset (Difference)')
        axs[1, 1].set_xlabel('Time')
        axs[1, 1].set_ylabel('Value')
        axs[1, 1].legend()

        axs[2, 1].plot(test_results['Test Predictions'][test:test+window_test] - test_results['Actuals'][test:test+window_test], label='Difference', color='red') #
        axs[2, 1].set_title('Testing dataset (Difference)')
        axs[2, 1].set_xlabel('Time')
        axs[2, 1].set_ylabel('Value')
        axs[2, 1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()
        
    elif which_dataset=='train':
        # Create a figure and subplots for testing dataset
        fig, axs = plt.subplots(
            2, 1,               # 2 rows, 1 column
            figsize=(20, 5),    # Width = 20 , height = 5 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0].plot(train_results['Train Predictions'][train:train+window_train], label='Model')
        axs[0].plot(train_results['Actuals'][train:train+window_train], label='Real')
        axs[0].set_title('Training dataset')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Value')
        axs[0].legend()

        # Plot difference between model and real data
        axs[1].plot(train_results['Train Predictions'][train:train+window_train] - train_results['Actuals'][train:train+window_train], label='Difference', color='red')
        axs[1].set_title('Training dataset (Difference)')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel('Value')
        axs[1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()

    elif which_dataset=='val':
        # Create a figure and subplots for testing dataset
        fig, axs = plt.subplots(
            2, 1,               # 2 rows, 1 column
            figsize=(20, 5),    # Width = 20 , height = 5 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0].plot(val_results['Val Predictions'][val:val+window_val], label='Model')
        axs[0].plot(val_results['Actuals'][val:val+window_val], label='Real')
        axs[0].set_title('Validation dataset')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Value')
        axs[0].legend()

        # Plot difference between model and real data
        axs[1].plot(val_results['Val Predictions'][val:val+window_val] - val_results['Actuals'][val:val+window_val], label='Difference', color='red')
        axs[1].set_title('Validation dataset (Difference)')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel('Value')
        axs[1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()
    
    elif which_dataset=='test':
        # Create a figure and subplots for testing dataset
        fig, axs = plt.subplots(
            2, 1,               # 2 rows, 1 column
            figsize=(20, 5),    # Width = 20 , height = 5 
            gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
        )

        # Plot data on subplots
        axs[0].plot(test_results['Test Predictions'][test:test+window_test], label='Model')
        axs[0].plot(test_results['Actuals'][test:test+window_test], label='Real')
        axs[0].set_title('Testing dataset')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Value')
        axs[0].legend()

        # Plot difference between model and real data
        axs[1].plot(test_results['Test Predictions'][test:test+window_test] - test_results['Actuals'][test:test+window_test], label='Difference', color='red')
        axs[1].set_title('Testing dataset (Difference)')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel('Value')
        axs[1].legend()

        fig.suptitle(plot_name, fontsize=16)
        plt.show()

    else: print("Please enter a valid dataset and parameters to display!")

In [ ]:
#Funkcja odpowiedzialna za wyświetlanie wyników z testing dataset
def display_test_results(test_results, plot_name):
    # Create a figure and subplots for the test dataset
    fig, axs = plt.subplots(
        2, 1,               # 2 rows, 1 column
        figsize=(20, 10),    # Width = 20, height = 10
        gridspec_kw=dict(hspace=0.5)   # Vertical spacing between subplots
    )

    # Plot predictions vs actuals
    axs[0].plot(test_results['Test Predictions'], label='Model Predictions')
    axs[0].plot(test_results['Actuals'], label='Actual Values')
    axs[0].set_title('Testing Dataset Predictions vs Actuals')
    axs[0].set_xlabel('Time')
    axs[0].set_ylabel('Value')
    axs[0].legend()

    # Plot the difference between predictions and actuals
    axs[1].plot(test_results['Test Predictions'] - test_results['Actuals'], label='Prediction Error', color='red')
    axs[1].set_title('Testing Dataset Prediction Error')
    axs[1].set_xlabel('Time')
    axs[1].set_ylabel('Value')
    axs[1].legend()

    # Set the main title and show the plot
    fig.suptitle(plot_name, fontsize=20)
    plt.show()


In [ ]:
#Funkcja odpowiedzialna za zapisywanie dataframe do csv
def save(df, file_name):
  df.to_csv('predictions_new/'+file_name+'.csv')
  

In [ ]:
#Funkcja odpowiedzialna za odczytywanie dataframe z csv
def get(file_name):
  return pd.read_csv('predictions_new/'+file_name+'.csv')

In [ ]:
# #Porównanie wykresów
# #Funkcja odpowedzialna za wyświetlanie wyników
# def display_comparison(train_results, val_results, test_results, plot_name):
#         # Create a figure and subplots for each dataset from cross validation
#         fig, axs = plt.subplots(
#             3, 2,               # 3 rows, 2 columns
#             figsize=(20, 16),    # Width = 10 , height = 8 
#             gridspec_kw=dict(hspace=0.5, wspace=0.5)   # Spacing between subplots
#         )

#         # Plot data on subplots
#         axs[0, 0].plot(train_results['Test Predictions'], label='Model')
#         axs[0, 0].plot(train_results['Actuals'], label='Real')
#         axs[0, 0].set_title('Testing dataset LSTM')
#         axs[0, 0].set_xlabel('Time')
#         axs[0, 0].set_ylabel('Value')
#         axs[0, 0].legend()

#         axs[1, 0].plot(val_results['Test Predictions'], label='Model')
#         axs[1, 0].plot(val_results['Actuals'], label='Real')
#         axs[1, 0].set_title('Testing dataset MLP ARX')
#         axs[1, 0].set_xlabel('Time')
#         axs[1, 0].set_ylabel('Value')
#         axs[1, 0].legend()

#         axs[2, 0].plot(test_results['Test Predictions'], label='Model')
#         axs[2, 0].plot(test_results['Actuals'], label='Real')
#         axs[2, 0].set_title('Testing dataset LINEAR ARX')
#         axs[2, 0].set_xlabel('Time')
#         axs[2, 0].set_ylabel('Value')
#         axs[2, 0].legend()

#         # Plot difference between model and real data
#         axs[0, 1].plot(train_results['Test Predictions'] - train_results['Actuals'], label='Difference', color='red')
#         axs[0, 1].set_title('Testing dataset LSTM (Difference)')
#         axs[0, 1].set_xlabel('Time')
#         axs[0, 1].set_ylabel('Value')
#         axs[0, 1].legend()

#         axs[1, 1].plot(val_results['Test Predictions']- val_results['Actuals'], label='Difference', color='red' )
#         axs[1, 1].set_title('Testing dataset MLP ARX (Difference)')
#         axs[1, 1].set_xlabel('Time')
#         axs[1, 1].set_ylabel('Value')
#         axs[1, 1].legend()

#         axs[2, 1].plot(test_results['Test Predictions'] - test_results['Actuals'], label='Difference', color='red')
#         axs[2, 1].set_title('Testing dataset LINEAR ARX (Difference)')
#         axs[2, 1].set_xlabel('Time')
#         axs[2, 1].set_ylabel('Value')
#         axs[2, 1].legend()

#         fig.suptitle(plot_name, fontsize=16)
#         plt.show()
        

In [ ]:
#Porównanie wykresów + jednostki

def display_comparison(train_results, val_results, test_results, plot_name, y_unit='°C', x_unit='100s'):
    # Create a figure and subplots for each dataset from cross validation
    fig, axs = plt.subplots(
        3, 2,               
        figsize=(20, 16),    
        gridspec_kw=dict(hspace=0.5, wspace=0.5)   
    )

    # Plot data on subplots
    axs[0, 0].plot(train_results['Test Predictions'], label='Model')
    axs[0, 0].plot(train_results['Actuals'], label='Real')
    axs[0, 0].set_title('Testing dataset LSTM')
    axs[0, 0].set_xlabel(f'Time [{x_unit}]')
    axs[0, 0].set_ylabel(f'Value [{y_unit}]')
    axs[0, 0].legend()

    axs[1, 0].plot(val_results['Test Predictions'], label='Model')
    axs[1, 0].plot(val_results['Actuals'], label='Real')
    axs[1, 0].set_title('Testing dataset MLP ARX')
    axs[1, 0].set_xlabel(f'Time [{x_unit}]')
    axs[1, 0].set_ylabel(f'Value [{y_unit}]')
    axs[1, 0].legend()

    axs[2, 0].plot(test_results['Test Predictions'], label='Model')
    axs[2, 0].plot(test_results['Actuals'], label='Real')
    axs[2, 0].set_title('Testing dataset LINEAR ARX')
    axs[2, 0].set_xlabel(f'Time [{x_unit}]')
    axs[2, 0].set_ylabel(f'Value [{y_unit}]')
    axs[2, 0].legend()

    # Plot difference between model and real data
    axs[0, 1].plot(train_results['Test Predictions'] - train_results['Actuals'], label='Difference', color='red')
    axs[0, 1].set_title('Testing dataset LSTM (Difference)')
    axs[0, 1].set_xlabel(f'Time [{x_unit}]')
    axs[0, 1].set_ylabel(f'Difference [{y_unit}]')
    axs[0, 1].legend()

    axs[1, 1].plot(val_results['Test Predictions'] - val_results['Actuals'], label='Difference', color='red')
    axs[1, 1].set_title('Testing dataset MLP ARX (Difference)')
    axs[1, 1].set_xlabel(f'Time [{x_unit}]')
    axs[1, 1].set_ylabel(f'Difference [{y_unit}]')
    axs[1, 1].legend()

    axs[2, 1].plot(test_results['Test Predictions'] - test_results['Actuals'], label='Difference', color='red')
    axs[2, 1].set_title('Testing dataset LINEAR ARX (Difference)')
    axs[2, 1].set_xlabel(f'Time [{x_unit}]')
    axs[2, 1].set_ylabel(f'Difference [{y_unit}]')
    axs[2, 1].legend()

    fig.suptitle(plot_name, fontsize=16)
    plt.show()


In [ ]:
#Porównanie wykresów wyników jednego modelu (train, val, test) + jednostki

def display_train_val_test(train_results, val_results, test_results, plot_name, y_unit='°C', x_unit='100s'):
    # Create a figure and subplots for each dataset from cross validation
    fig, axs = plt.subplots(
        3, 2,               
        figsize=(20, 16),    
        gridspec_kw=dict(hspace=0.5, wspace=0.5)
    )

    # Plot data on subplots
    axs[0, 0].plot(train_results['Train Predictions'], label='Model')
    axs[0, 0].plot(train_results['Actuals'], label='Real')
    axs[0, 0].set_title('Training dataset')
    axs[0, 0].set_xlabel(f'Time [{x_unit}]')
    axs[0, 0].set_ylabel(f'Value [{y_unit}]')
    axs[0, 0].legend()

    axs[1, 0].plot(val_results['Val Predictions'], label='Model')
    axs[1, 0].plot(val_results['Actuals'], label='Real')
    axs[1, 0].set_title('Validation dataset')
    axs[1, 0].set_xlabel(f'Time [{x_unit}]')
    axs[1, 0].set_ylabel(f'Value [{y_unit}]')
    axs[1, 0].legend()

    axs[2, 0].plot(test_results['Test Predictions'], label='Model')
    axs[2, 0].plot(test_results['Actuals'], label='Real')
    axs[2, 0].set_title('Testing dataset')
    axs[2, 0].set_xlabel(f'Time [{x_unit}]')
    axs[2, 0].set_ylabel(f'Value [{y_unit}]')
    axs[2, 0].legend()

    # Plot difference between model and real data
    axs[0, 1].plot(train_results['Train Predictions'] - train_results['Actuals'], label='Difference', color='red')
    axs[0, 1].set_title('Training dataset (Difference)')
    axs[0, 1].set_xlabel(f'Time [{x_unit}]')
    axs[0, 1].set_ylabel(f'Difference [{y_unit}]')
    axs[0, 1].legend()

    axs[1, 1].plot(val_results['Val Predictions'] - val_results['Actuals'], label='Difference', color='red')
    axs[1, 1].set_title('Validation dataset (Difference)')
    axs[1, 1].set_xlabel(f'Time [{x_unit}]')
    axs[1, 1].set_ylabel(f'Difference [{y_unit}]')
    axs[1, 1].legend()

    axs[2, 1].plot(test_results['Test Predictions'] - test_results['Actuals'], label='Difference', color='red')
    axs[2, 1].set_title('Testing dataset (Difference)')
    axs[2, 1].set_xlabel(f'Time [{x_unit}]')
    axs[2, 1].set_ylabel(f'Difference [{y_unit}]')
    axs[2, 1].legend()

    fig.suptitle(plot_name, fontsize=16)
    plt.show()


In [ ]:
def calculate_metrics(y_true, y_pred):
    me = (y_true - y_pred).mean()
    mse = ((y_true - y_pred) ** 2).mean()
    mae = (y_true - y_pred).abs().mean()
    rmse = np.sqrt(mse)
    return me, mse, mae, rmse

In [ ]:
def highlight_min_green(s):
    is_min = s == s.min()
    return ['background-color: lightgreen' if v else '' for v in is_min]

# DATA PREPARATION AND METHODS MLP ARX AND LINEAR ARX

In [ ]:
#imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
#Resampling and adding shifted output values as inputs
#Testing data
test_file_data_MLP = test_file_data.iloc[::10]
test_file_data_MLP.reset_index(inplace= True, drop= True)

test_file_data_MLP['y_k_1'] = test_file_data_MLP[col_WS1[0]].shift(1)  # y[k-1]
test_file_data_MLP['y_k_2'] = test_file_data_MLP[col_WS1[0]].shift(2)  # y[k-2]
test_file_data_MLP = test_file_data_MLP.dropna().reset_index(drop=True)

#Validation data
val_file_data_MLP = val_file_data.iloc[::10]
val_file_data_MLP.reset_index(inplace= True, drop= True)

val_file_data_MLP['y_k_1'] = val_file_data_MLP[col_WS1[0]].shift(1)  # y[k-1]
val_file_data_MLP['y_k_2'] = val_file_data_MLP[col_WS1[0]].shift(2)  # y[k-2]
val_file_data_MLP = val_file_data_MLP.dropna().reset_index(drop=True)

#Training data
train_file_data_MLP = train_file_data.iloc[::10]
train_file_data_MLP.reset_index(inplace= True, drop= True)

train_file_data_MLP['y_k_1'] = train_file_data_MLP[col_WS1[0]].shift(1)  # y[k-1]
train_file_data_MLP['y_k_2'] = train_file_data_MLP[col_WS1[0]].shift(2)  # y[k-2]
train_file_data_MLP = train_file_data_MLP.dropna().reset_index(drop=True)

In [ ]:
#Funkcja odpowiedzialna za tworzenie modelu MLP
def create_model_MLP(var_num):
    model = Sequential()
    model.add(Dense(64, activation = 'relu', input_dim = var_num))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(1, activation = 'linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['rmse'])
    return model

In [ ]:
#Funkcja odpowiedzialna za przewidywanie kolejnych wartości MLP ARX-> wersja ze średnią ważoną
def predict_MLP(model, input, num_steps, labels):
    predictions = np.zeros(num_steps) #empty numpy array for storing predictions
    input = np.array(input) #We create a temporary variable for input
    for i in range(num_steps):
        
        #The data needs to be reshaped before passing into the model so that it fits earlier defined input sizes
        next_value = model.predict(input[i].reshape(1,input.shape[1]), verbose=0) 
        
        #Adding predicted values to predictions array
        predictions[i] = next_value[0][0]
        
        #Switching y[k-1] and y[k-2] values for those predicted by the model
        if i<num_steps-1:
           input[i+1][input.shape[1]-2] = ((9 * predictions[i] + labels[i]) / 10) #y[k-1] one before last column
           if i!=0:
               input[i+1][input.shape[1]-1] = ((9 * predictions[i-1] + labels[i-1]) / 10)#y[k-2] last column
           else:
               pass
        
        
        #Counting how many steps forward have already been predicted and printing it to the console
        if i % 500 == 0:
            #os.system('cls')
            print(i, '/', num_steps)
    print("Done")
    return predictions


In [ ]:
# #Funkcja odpowiedzialna za przewidywanie kolejnych wartości MLP ARX -> wersja bez średniej ważonej
# def predict_MLP(model, input, num_steps):
#     predictions = np.zeros(num_steps) #empty numpy array for storing predictions
#     input = np.array(input) #We create a temporary variable for input
    
#     for i in range(num_steps):
        
#         #The data needs to be reshaped before passing into the model so that it fits earlier defined input sizes
#         next_value = model.predict(input[i].reshape(1,input.shape[1]), verbose=0) 
        
#         #Adding predicted values to predictions array
#         predictions[i] = next_value[0][0]
        
#         #Switching y[k-1] and y[k-2] values for those predicted by the model
#         if i<num_steps-1:
#            input[i+1][input.shape[1]-2] = predictions[i]#y[k-1] one before last column
#            if i!=0:
#                input[i+1][input.shape[1]-1] = predictions[i-1]#y[k-2] last column
#            else:
#                pass
        
#         #Counting how many steps forward have already been predicted and printing it to the console
#         if i % 500 == 0:
#             #os.system('cls')
#             print(i, '/', num_steps)
#     print("Done")
#     return predictions


In [ ]:
#Funkcja odpowiedzialna za przewidywanie kolejnych wartości LINEAR ARX-> wersja ze średnią ważoną
def predict_LINEAR(model, input, num_steps, labels):
    predictions = np.zeros(num_steps) #empty numpy array for storing predictions
    input = np.array(input) #We create a temporary variable for input
    
    for i in range(num_steps):
        
        #The data needs to be reshaped before passing into the model so that it fits earlier defined input sizes
        next_value = model.predict(input[i].reshape(1,input.shape[1])) 
        
        #Adding predicted values to predictions array
        predictions[i] = next_value[0]
        
        #Switching y[k-1] and y[k-2] values for those predicted by the model
    
        if i<num_steps-1: 
           input[i+1][input.shape[1]-2] = ((9 * predictions[i] + labels[i]) / 10) #y[k-1] one before last column
           if i!=0:
               input[i+1][input.shape[1]-1] = ((9 * predictions[i-1] + labels[i-1]) / 10)#y[k-2] last column
           else:
               pass
        
        #Counting how many steps forward have already been predicted and printing it to the console
        if i % 500 == 0:
            #os.system('cls')
            print(i, '/', num_steps)
    print("Done")
    return predictions

# MODEL 1 - Model obrotów wentylatora WS1

In [ ]:
#Comparison
display_comparison(get('test_results_WS1_LSTM'), get('test_results_WS1_MLP'), get('test_results_WS1_LINEAR'), "MODEL 1 - Model obrotów wentylatora WS1", y_unit='rpm')

In [ ]:
metrics = []

## LSTM

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  df_WS1 = file_data[col_WS1]
  
  X_train, y_train = data_to_X_y(df_WS1[col_WS1[0]], WINDOW_SIZE)
  var_train = df_WS1[col_WS1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train] 
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data[col_WS1[0]], WINDOW_SIZE)
var_val = val_file_data[col_WS1[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]


In [ ]:
#Model creation
model1 = create_model(9) #9 additional variables not including sequences

cp = ModelCheckpoint('model_WS1/model_WS1.keras', save_best_only=True)

model1.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model1.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model1 = load_model('model_WS1/model_WS1.keras')

X_test, y_test = data_to_X_y(test_file_data[col_WS1[0]], WINDOW_SIZE)
var_test = test_file_data[col_WS1[1:]][WINDOW_SIZE-1:].to_numpy()
#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]

test = 0
window_test = X_test.shape[0]
# train = 0
# window_train = X_train.shape[0]
# val = 0
# window_val = X_val.shape[0]


# train_predictions = predict(model1, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_WS1 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(denorm(train_results_WS1, col_WS1[0]),'train_results_WS1')
# save(train_results_WS1,'train_results_WS1_normalized')

# val_predictions = predict(model1, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_WS1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(denorm(val_results_WS1, col_WS1[0]),'val_results_WS1')
# save(val_results_WS1,'val_results_WS1_normalized')

test_predictions = predict(model1, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
test_results_WS1 = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_WS1, col_WS1[0]), 'test_results_WS1')
save(test_results_WS1,'test_results_WS1_normalized')


In [ ]:
#Displaying results
display_test_results(get('test_results_WS1'), "MODEL 1 - Model obrotów wentylatora WS1 LSTM")
#display(get('train_results_WS1'), get('val_results_WS1'), get('test_results_WS1'),'all', "MODEL 1 - Model obrotów wentylatora WS1 LSTM")

## LSTM resampled data

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  df_WS1 = file_data[col_WS1]
  
  X_train, y_train = data_to_X_y(df_WS1[col_WS1[0]], WINDOW_SIZE)
  var_train = df_WS1[col_WS1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train] 
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data_resampled[col_WS1[0]], WINDOW_SIZE)
var_val = val_file_data_resampled[col_WS1[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_WS1 = file_data[col_WS1]
  
  X_test, y_test = data_to_X_y(df_WS1[col_WS1[0]], WINDOW_SIZE)
  var_test = df_WS1[col_WS1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_test = min(X_test.shape[0],min(y_test.shape[0], var_test.shape[0]))
  X_test = X_test[:length_test] 
  y_test = y_test[:length_test] 
  var_test = var_test[:length_test] 
  
  test_data.append([X_test, y_test, var_test])

In [ ]:
#Model creation
model1 = create_model(9) #9 additional variables not including sequences

cp = ModelCheckpoint('model_WS1/model_WS1_LSTM.keras', save_best_only=True)

model1.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model1.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=20, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model1 = load_model('model_WS1/model_WS1_LSTM.keras')
train_results = []
for [X_train, y_train, var_train] in prepared_data:
  #We specify from which point we want to predict the next values for each dataset [s]
  #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict(model1, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)

train_results_WS1 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_WS1, col_WS1[0]), 'train_results_WS1_LSTM')
save(train_results_WS1,'train_results_WS1_normalized_LSTM')

val = 0
window_val = X_val.shape[0]
val_predictions = predict(model1, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val, y_val)
val_results_WS1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_WS1, col_WS1[0]),'val_results_WS1_LSTM')
save(val_results_WS1,'val_results_WS1_normalized_LSTM')

test_results = []
for [X_test, y_test, var_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict(model1, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)

test_results_WS1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_WS1, col_WS1[0]), 'test_results_WS1_LSTM')
save(test_results_WS1,'test_results_WS1_normalized_LSTM')

In [ ]:
#Displaying results
display_test_results(get('test_results_WS1_LSTM_test'), "MODEL 1 - Model obrotów wentylatora WS1 LSTM")

In [ ]:
display_train_val_test(get('train_results_WS1_LSTM'), get('val_results_WS1_LSTM'), get('test_results_WS1_LSTM'), "MODEL 1 LSTM - Model obrotów wentylatora WS1", y_unit='rpm')

## MLP ARX and LINEAR ARX resampled data

In [ ]:
#Data preparation
#Przetrwarzamy dane tak, żeby można było przekazać dwa poprzednie kroki zmiennej jako parametry do naszego modelu

prepared_data = []
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  #Taking only the columns relevant for the model
  df_WS1 = file_data[col_WS1]
  
  # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_WS1['y_k_1'] = df_WS1[col_WS1[0]].shift(1)  # y[k-1]
  df_WS1['y_k_2'] = df_WS1[col_WS1[0]].shift(2)  # y[k-2]
  df_WS1 = df_WS1.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_train = df_WS1[col_WS1[1:] + ['y_k_1','y_k_2']]
  y_train = df_WS1[col_WS1[0]]
  
  prepared_data.append([X_train, y_train])
  
X_val = val_file_data_MLP[col_WS1[1:] + ['y_k_1','y_k_2']]
y_val = val_file_data_MLP[col_WS1[0]]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_WS1 = file_data[col_WS1]
  
    # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_WS1['y_k_1'] = df_WS1[col_WS1[0]].shift(1)  # y[k-1]
  df_WS1['y_k_2'] = df_WS1[col_WS1[0]].shift(2)  # y[k-2]
  df_WS1 = df_WS1.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_test = df_WS1[col_WS1[1:] + ['y_k_1','y_k_2']]
  y_test = df_WS1[col_WS1[0]]
  
  test_data.append([X_test, y_test])



In [ ]:
#Model creation MLP
model1 = create_model_MLP(11) #11 additional variables (9 variables + 2 earlier model outputs y[k-1] and y[k-2])

cp = ModelCheckpoint('model_WS1/model_WS1_MLP.keras', save_best_only=True)

model1.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model1 = load_model('model_WS1/model_WS1_MLP.keras')

train_results = []
for [X_train, y_train] in prepared_data:
  # #We specify from which point we want to predict the next values for each dataset [s]
  # #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict_MLP(model1, X_train, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)
  
train_results_WS1 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_WS1, col_WS1[0]), 'train_results_WS1_MLP')
save(train_results_WS1,'train_results_WS1_normalized_MLP')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_MLP(model1, X_val, window_val, y_val)
val_results_WS1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_WS1, col_WS1[0]),'val_results_WS1_MLP')
save(val_results_WS1,'val_results_WS1_normalized_MLP')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_MLP(model1, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_WS1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_WS1, col_WS1[0]), 'test_results_WS1_MLP')
save(test_results_WS1,'test_results_WS1_normalized_MLP')

In [ ]:
#LINEAR
#Model creation linear
model1 = LinearRegression()
#Model training linear
X_train = train_file_data_MLP[col_WS1[1:] + ['y_k_1','y_k_2']]
y_train = train_file_data_MLP[col_WS1[0]]

model1.fit(X_train, y_train)

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
train_predictions = predict_LINEAR(model1, X_train, window_train, y_train)
train_results_WS1 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_WS1, col_WS1[0]),'train_results_WS1_LINEAR')
save(train_results_WS1,'train_results_WS1_normalized_LINEAR')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_LINEAR(model1, X_val, window_val, y_val)
val_results_WS1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_WS1, col_WS1[0]),'val_results_WS1_LINEAR')
save(val_results_WS1,'val_results_WS1_normalized_LINEAR')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_LINEAR(model1, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_WS1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_WS1, col_WS1[0]), 'test_results_WS1_LINEAR')
save(test_results_WS1,'test_results_WS1_normalized_LINEAR')


In [ ]:
#Displaying results MLP
display_test_results(get('test_results_WS1_MLP'), "MODEL 1 - Model obrotów wentylatora WS1 MLP ARX")

In [ ]:
display_train_val_test(get('train_results_WS1_MLP'), get('val_results_WS1_MLP'), get('test_results_WS1_MLP'), "MODEL 1 MLP ARX - Model obrotów wentylatora WS1", y_unit='rpm')

In [ ]:
#Displaying results LINEAR
display_test_results(get('test_results_WS1_LINEAR'), "MODEL 1 - Model obrotów wentylatora WS1 LINEAR ARX")

In [ ]:
display_train_val_test(get('train_results_WS1_LINEAR'), get('val_results_WS1_LINEAR'), get('test_results_WS1_LINEAR'), "MODEL 1 LINEAR ARX - Model obrotów wentylatora WS1", y_unit='rpm')

## Metrics

In [ ]:
metrics = []
files = ["train_results_WS1_normalized_LSTM", "val_results_WS1_normalized_LSTM", "test_results_WS1_normalized_LSTM", "train_results_WS1_normalized_MLP", "val_results_WS1_normalized_MLP", "test_results_WS1_normalized_MLP", "train_results_WS1_normalized_LINEAR", "val_results_WS1_normalized_LINEAR", "test_results_WS1_normalized_LINEAR"]
for file in files:
  results = get(file).drop(columns=["Unnamed: 0"])
  me, mse, mae, rmse = calculate_metrics(results[results.columns[1]], results[results.columns[0]])#Actuals and then predicitons
  metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df['Model'] = ['LSTM train', 'LSTM val', 'LSTM test', 'MLP ARX train', 'MLP ARX val', 'MLP ARX test', 'LINEAR ARX train', 'LINEAR ARX val', 'LINEAR ARX test']
metrics_df.rename(columns={'Model': 'MODEL 1 - Model obrotów wentylatora WS1', 'ME': 'Mean_Error', 'MSE': 'Mean_Squared_Error', 
                           'MAE': 'Mean_Absolute_Error', 'RMSE': 'Root_Mean_Squared_Error'}, inplace=True)
metrics_df.to_csv("metrics/WS1_models_metrics_NEW.csv", index=False)


In [ ]:
metrics_df = pd.read_csv("metrics/WS1_models_metrics_NEW.csv")
metrics_df.iloc[6:, [0, 2, 4]].style.apply(highlight_min_green, subset=metrics_df.iloc[:, [ 2, 4]].columns)#['Mean_Squared_Error', 'Root_Mean_Squared_Error'], axis=0)

# MODEL 2 - Model obrotów wentylatora WS2

In [ ]:
#Comparison
display_comparison(get('test_results_WS2_LSTM'), get('test_results_WS2_MLP'), get('test_results_WS2_LINEAR'), "MODEL 2 - Model obrotów wentylatora WS2", y_unit='rpm')

In [ ]:
metrics = []

## LSTM

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  df_WS2 = file_data[col_WS2]
  
  X_train, y_train = data_to_X_y(df_WS2[col_WS2[0]], WINDOW_SIZE)
  var_train = df_WS2[col_WS2[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data[col_WS2[0]], WINDOW_SIZE)
var_val = val_file_data[col_WS2[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]


In [ ]:
#Model creation
model2 = create_model(10) #10 additional variables not including sequences

cp = ModelCheckpoint('model_WS2/model_WS2.keras', save_best_only=True)

model2.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model2.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model predictions
model2 = load_model('model_WS2/model_WS2.keras')

X_test, y_test = data_to_X_y(test_file_data[col_WS2[0]], WINDOW_SIZE)
var_test = test_file_data[col_WS2[1:]][WINDOW_SIZE-1:].to_numpy()
#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]

test = 0
window_test = X_test.shape[0]
# train = 0
# window_train = X_train.shape[0]
# val = 0
# window_val = X_val.shape[0]


# train_predictions = predict(model2, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_WS2 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(denorm(train_results_WS2, col_WS2[0]),'train_results_WS2')
# save(train_results_WS2,'train_results_WS2_normalized')

# val_predictions = predict(model2, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_WS2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(denorm(val_results_WS2, col_WS2[0]),'val_results_WS2')
# save(val_results_WS2,'val_results_WS2_normalized')

test_predictions = predict(model2, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test)
test_results_WS2 = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_WS2, col_WS2[0]), 'test_results_WS2')
save(test_results_WS2,'test_results_WS2_normalized')

In [ ]:
#Displaying results
display_test_results(get('test_results_WS2'), "MODEL 2 - Model obrotów wentylatora WS2 LSTM")
#display(get('train_results_WS2'), get('val_results_WS2'), get('test_results_WS2'),'all', "MODEL 2 - Model obrotów wentylatora WS2 LSTM")

## LSTM resampled data

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
   #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  df_WS2 = file_data[col_WS2]
  
  X_train, y_train = data_to_X_y(df_WS2[col_WS2[0]], WINDOW_SIZE)
  var_train = df_WS2[col_WS2[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data_resampled[col_WS2[0]], WINDOW_SIZE)
var_val = val_file_data_resampled[col_WS2[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_WS2 = file_data[col_WS2]
  
  X_test, y_test = data_to_X_y(df_WS2[col_WS2[0]], WINDOW_SIZE)
  var_test = df_WS2[col_WS2[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_test = min(X_test.shape[0],min(y_test.shape[0], var_test.shape[0]))
  X_test = X_test[:length_test] 
  y_test = y_test[:length_test] 
  var_test = var_test[:length_test] 
  
  test_data.append([X_test, y_test, var_test])


In [ ]:
#Model creation
model2 = create_model(10) #10 additional variables not including sequences

cp = ModelCheckpoint('model_WS2/model_WS2_LSTM.keras', save_best_only=True)

model2.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model2.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model2 = load_model('model_WS2/model_WS2_LSTM.keras')
train_results = []
for [X_train, y_train, var_train] in prepared_data:
  #We specify from which point we want to predict the next values for each dataset [s]
  #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict(model2, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)

train_results_WS2 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_WS2, col_WS2[0]), 'train_results_WS2_LSTM')
save(train_results_WS2,'train_results_WS2_normalized_LSTM')

val = 0
window_val = X_val.shape[0]
val_predictions = predict(model2, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val, y_val)
val_results_WS2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_WS2, col_WS2[0]),'val_results_WS2_LSTM')
save(val_results_WS2,'val_results_WS2_normalized_LSTM')

test_results = []
for [X_test, y_test, var_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict(model2, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)

test_results_WS2 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_WS2, col_WS2[0]), 'test_results_WS2_LSTM')
save(test_results_WS2,'test_results_WS2_normalized_LSTM')

In [ ]:
#Displaying results
display_test_results(get('test_results_WS2_LSTM'), "MODEL 2 - Model obrotów wentylatora WS2 LSTM")

In [ ]:
display_train_val_test(get('train_results_WS2_LSTM'), get('val_results_WS2_LSTM'), get('test_results_WS2_LSTM'), "MODEL 2 LSTM - Model obrotów wentylatora WS2", y_unit='rpm')

## MLP ARX and LINEAR ARX resampled data

In [ ]:
#Data preparation
#Przetrwarzamy dane tak, żeby można było przekazać dwa poprzednie kroki zmiennej jako parametry do naszego modelu

prepared_data = []
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  #Taking only the columns relevant for the model
  df_WS2 = file_data[col_WS2]
  
  # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_WS2['y_k_1'] = df_WS2[col_WS2[0]].shift(1)  # y[k-1]
  df_WS2['y_k_2'] = df_WS2[col_WS2[0]].shift(2)  # y[k-2]
  df_WS2 = df_WS2.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_train = df_WS2[col_WS2[1:] + ['y_k_1','y_k_2']]
  y_train = df_WS2[col_WS2[0]]
  
  prepared_data.append([X_train, y_train])
  
X_val = val_file_data_MLP[col_WS2[1:] + ['y_k_1','y_k_2']]
y_val = val_file_data_MLP[col_WS2[0]]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_WS2 = file_data[col_WS2]
  
    # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_WS2['y_k_1'] = df_WS2[col_WS2[0]].shift(1)  # y[k-1]
  df_WS2['y_k_2'] = df_WS2[col_WS2[0]].shift(2)  # y[k-2]
  df_WS2 = df_WS2.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_test = df_WS2[col_WS2[1:] + ['y_k_1','y_k_2']]
  y_test = df_WS2[col_WS2[0]]
  
  test_data.append([X_test, y_test])


In [ ]:
#Model creation
model2 = create_model_MLP(12) #12  variables (10 variables + 2 earlier model outputs y[k-1] and y[k-2])

cp = ModelCheckpoint('model_WS2/model_WS2_MLP.keras', save_best_only=True)

model2.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model2 = load_model('model_WS2/model_WS2_MLP.keras')

train_results = []
for [X_train, y_train] in prepared_data:
  # #We specify from which point we want to predict the next values for each dataset [s]
  # #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict_MLP(model2, X_train, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)
  
train_results_WS2 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_WS2, col_WS2[0]), 'train_results_WS2_MLP')
save(train_results_WS2,'train_results_WS2_normalized_MLP')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_MLP(model2, X_val, window_val, y_val)
val_results_WS2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_WS2, col_WS2[0]),'val_results_WS2_MLP')
save(val_results_WS2,'val_results_WS2_normalized_MLP')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_MLP(model2, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_WS2 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_WS2, col_WS2[0]), 'test_results_WS2_MLP')
save(test_results_WS2,'test_results_WS2_normalized_MLP')

In [ ]:
#LINEAR
#Model creation linear
model2 = LinearRegression()
#Model training linear
X_train = train_file_data_MLP[col_WS2[1:] + ['y_k_1','y_k_2']]
y_train = train_file_data_MLP[col_WS2[0]]

model2.fit(X_train, y_train)

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
train_predictions = predict_LINEAR(model2, X_train, window_train, y_train)
train_results_WS2 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_WS2, col_WS2[0]),'train_results_WS2_LINEAR')
save(train_results_WS2,'train_results_WS2_normalized_LINEAR')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_LINEAR(model2, X_val, window_val, y_val)
val_results_WS2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_WS2, col_WS2[0]),'val_results_WS2_LINEAR')
save(val_results_WS2,'val_results_WS2_normalized_LINEAR')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_LINEAR(model2, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_WS2 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_WS2, col_WS2[0]), 'test_results_WS2_LINEAR')
save(test_results_WS2,'test_results_WS2_normalized_LINEAR')


In [ ]:
#Displaying results MLP
display_test_results(get('test_results_WS2_MLP'), "MODEL 2 - Model obrotów wentylatora WS2 MLP ARX")

In [ ]:
display_train_val_test(get('train_results_WS2_MLP'), get('val_results_WS2_MLP'), get('test_results_WS2_MLP'), "MODEL 2 MLP ARX - Model obrotów wentylatora WS2", y_unit='rpm')

In [ ]:
#Displaying results LINEAR
display_test_results(get('test_results_WS2_LINEAR'), "MODEL 2 - Model obrotów wentylatora WS2 LINEAR ARX")

In [ ]:
display_train_val_test(get('train_results_WS2_LINEAR'), get('val_results_WS2_LINEAR'), get('test_results_WS2_LINEAR'), "MODEL 2 LINEAR ARX - Model obrotów wentylatora WS2", y_unit='rpm')

## Metrics

In [ ]:
metrics = []
files = ["train_results_WS2_normalized_LSTM", "val_results_WS2_normalized_LSTM", "test_results_WS2_normalized_LSTM", "train_results_WS2_normalized_MLP", "val_results_WS2_normalized_MLP", "test_results_WS2_normalized_MLP", "train_results_WS2_normalized_LINEAR", "val_results_WS2_normalized_LINEAR", "test_results_WS2_normalized_LINEAR"]
for file in files:
  results = get(file).drop(columns=["Unnamed: 0"])
  me, mse, mae, rmse = calculate_metrics(results[results.columns[1]], results[results.columns[0]])#Actuals and then predicitons
  metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df['Model'] = ['LSTM train', 'LSTM val', 'LSTM test', 'MLP ARX train', 'MLP ARX val', 'MLP ARX test', 'LINEAR ARX train', 'LINEAR ARX val', 'LINEAR ARX test']
metrics_df.rename(columns={'Model': 'MODEL 2 - Model obrotów wentylatora WS2', 'ME': 'Mean_Error', 'MSE': 'Mean_Squared_Error', 
                           'MAE': 'Mean_Absolute_Error', 'RMSE': 'Root_Mean_Squared_Error'}, inplace=True)
metrics_df.to_csv("metrics/WS2_models_metrics_NEW.csv", index=False)

In [ ]:
metrics_df = pd.read_csv("metrics/WS2_models_metrics_NEW.csv")
metrics_df.iloc[6:, [0, 2, 4]].style.apply(highlight_min_green, subset=metrics_df.iloc[:, [ 2, 4]].columns)#['Mean_Squared_Error', 'Root_Mean_Squared_Error'], axis=0)

# MODEL 3 - Model pomiaru ciśnienia KP1 w komorze paleniskowej

In [ ]:
#Comparison
display_comparison(get('test_results_KP1_LSTM'), get('test_results_KP1_MLP'), get('test_results_KP1_LINEAR'), "MODEL 3 - Model pomiaru ciśnienia KP1 w komorze paleniskowej", y_unit='-')

In [ ]:
metrics = []

## LSTM

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  df_KP1 = file_data[col_KP1]
  
  X_train, y_train = data_to_X_y(df_KP1[col_KP1[0]], WINDOW_SIZE)
  var_train = df_KP1[col_KP1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data[col_KP1[0]], WINDOW_SIZE)
var_val = val_file_data[col_KP1[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

In [ ]:
#Model creation
model3 = create_model(4) #4 additional variables not including sequences

cp = ModelCheckpoint('model_KP1/model_KP1.keras', save_best_only=True)

model3.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model3.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model predictions
model3 = load_model('model_KP1/model_KP1.keras')

X_test, y_test = data_to_X_y(test_file_data[col_KP1[0]], WINDOW_SIZE)
var_test = test_file_data[col_KP1[1:]][WINDOW_SIZE-1:].to_numpy()
#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]

test = 0
window_test = X_test.shape[0]
# train = 0
# window_train = X_train.shape[0]
# val = 0
# window_val = X_val.shape[0]


# train_predictions = predict(model3, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_KP1 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(denorm(train_results_KP1, col_KP1[0]),'train_results_KP1')
# save(train_results_KP1,'train_results_KP1_normalized')

# val_predictions = predict(model3, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_KP1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(denorm(val_results_KP1, col_KP1[0]),'val_results_KP1')
# save(val_results_KP1,'val_results_KP1_normalized')

test_predictions = predict(model3, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test)
test_results_KP1 = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_KP1, col_KP1[0]), 'test_results_KP1')
save(test_results_KP1,'test_results_KP1_normalized')

In [ ]:
#Displaying results
display_test_results(get('test_results_KP1'), "MODEL 3 - Model pomiaru ciśnienia KP1 w komorze paleniskowej LSTM")
#display(get('train_results_KP1'), get('val_results_KP1'), get('test_results_KP1'),'all', "MODEL 3 - Model pomiaru ciśnienia KP1 w komorze paleniskowej LSTM")

## LSTM resampled data

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
   #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  df_KP1 = file_data[col_KP1]
  
  X_train, y_train = data_to_X_y(df_KP1[col_KP1[0]], WINDOW_SIZE)
  var_train = df_KP1[col_KP1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data_resampled[col_KP1[0]], WINDOW_SIZE)
var_val = val_file_data_resampled[col_KP1[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]


test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_KP1 = file_data[col_KP1]
  
  X_test, y_test = data_to_X_y(df_KP1[col_KP1[0]], WINDOW_SIZE)
  var_test = df_KP1[col_KP1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_test = min(X_test.shape[0],min(y_test.shape[0], var_test.shape[0]))
  X_test = X_test[:length_test] 
  y_test = y_test[:length_test] 
  var_test = var_test[:length_test] 
  
  test_data.append([X_test, y_test, var_test])

In [ ]:
#Model creation
model3 = create_model(4) #4 additional variables not including sequences

cp = ModelCheckpoint('model_KP1/model_KP1_LSTM.keras', save_best_only=True)

model3.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model3.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model3 = load_model('model_KP1/model_KP1_LSTM.keras')
train_results = []
for [X_train, y_train, var_train] in prepared_data:
  #We specify from which point we want to predict the next values for each dataset [s]
  #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict(model3, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)

train_results_KP1 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_KP1, col_KP1[0]), 'train_results_KP1_LSTM')
save(train_results_KP1,'train_results_KP1_normalized_LSTM')

val = 0
window_val = X_val.shape[0]
val_predictions = predict(model3, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val, y_val)
val_results_KP1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KP1, col_KP1[0]),'val_results_KP1_LSTM')
save(val_results_KP1,'val_results_KP1_normalized_LSTM')

test_results = []
for [X_test, y_test, var_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict(model3, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)

test_results_KP1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KP1, col_KP1[0]), 'test_results_KP1_LSTM')
save(test_results_KP1,'test_results_KP1_normalized_LSTM')

In [ ]:
#Displaying results
display_test_results(get('test_results_KP1_LSTM'), "MODEL 3 - Model pomiaru ciśnienia KP1 w komorze paleniskowej LSTM")


In [ ]:
display_train_val_test(get('train_results_KP1_LSTM'), get('val_results_KP1_LSTM'), get('test_results_KP1_LSTM'), "MODEL 3 LSTM - Model pomiaru ciśnienia KP1 w komorze paleniskowej", y_unit='-')

## MLP ARX and LINEAR ARX resampled data

In [ ]:
#Data preparation
#Przetrwarzamy dane tak, żeby można było przekazać dwa poprzednie kroki zmiennej jako parametry do naszego modelu

prepared_data = []
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  #Taking only the columns relevant for the model
  df_KP1 = file_data[col_KP1]
  
  # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_KP1['y_k_1'] = df_KP1[col_KP1[0]].shift(1)  # y[k-1]
  df_KP1['y_k_2'] = df_KP1[col_KP1[0]].shift(2)  # y[k-2]
  df_KP1 = df_KP1.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_train = df_KP1[col_KP1[1:] + ['y_k_1','y_k_2']]
  y_train = df_KP1[col_KP1[0]]
  
  prepared_data.append([X_train, y_train])
  
X_val = val_file_data_MLP[col_KP1[1:] + ['y_k_1','y_k_2']]
y_val = val_file_data_MLP[col_KP1[0]]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_KP1 = file_data[col_KP1]
  
    # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_KP1['y_k_1'] = df_KP1[col_KP1[0]].shift(1)  # y[k-1]
  df_KP1['y_k_2'] = df_KP1[col_KP1[0]].shift(2)  # y[k-2]
  df_KP1 = df_KP1.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_test = df_KP1[col_KP1[1:] + ['y_k_1','y_k_2']]
  y_test = df_KP1[col_KP1[0]]
  
  test_data.append([X_test, y_test])


In [ ]:
#Model creation
model3 = create_model_MLP(6) #6 variables (4+2y)

cp = ModelCheckpoint('model_KP1/model_KP1_MLP.keras', save_best_only=True)

model3.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model3.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model3 = load_model('model_KP1/model_KP1_MLP.keras')

train_results = []
for [X_train, y_train] in prepared_data:
  # #We specify from which point we want to predict the next values for each dataset [s]
  # #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict_MLP(model3, X_train, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)
  
train_results_KP1 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_KP1, col_KP1[0]), 'train_results_KP1_MLP')
save(train_results_KP1,'train_results_KP1_normalized_MLP')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_MLP(model3, X_val, window_val, y_val)
val_results_KP1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KP1, col_KP1[0]),'val_results_KP1_MLP')
save(val_results_KP1,'val_results_KP1_normalized_MLP')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_MLP(model3, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_KP1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KP1, col_KP1[0]), 'test_results_KP1_MLP')
save(test_results_KP1,'test_results_KP1_normalized_MLP')

In [ ]:
#LINEAR
#Model creation linear
model3 = LinearRegression()
#Model training linear
X_train = train_file_data_MLP[col_KP1[1:] + ['y_k_1','y_k_2']]
y_train = train_file_data_MLP[col_KP1[0]]

model3.fit(X_train, y_train)

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
train_predictions = predict_LINEAR(model3, X_train, window_train, y_train)
train_results_KP1 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_KP1, col_KP1[0]),'train_results_KP1_LINEAR')
save(train_results_KP1,'train_results_KP1_normalized_LINEAR')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_LINEAR(model3, X_val, window_val, y_val)
val_results_KP1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KP1, col_KP1[0]),'val_results_KP1_LINEAR')
save(val_results_KP1,'val_results_KP1_normalized_LINEAR')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_LINEAR(model3, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_KP1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KP1, col_KP1[0]), 'test_results_KP1_LINEAR')
save(test_results_KP1,'test_results_KP1_normalized_LINEAR')


In [ ]:
#Displaying results MLP
display_test_results(get('test_results_KP1_MLP'), "MODEL 3 - Model pomiaru ciśnienia KP1 w komorze paleniskowej MLP ARX")

In [ ]:
display_train_val_test(get('train_results_KP1_MLP'), get('val_results_KP1_MLP'), get('test_results_KP1_MLP'), "MODEL 3 MLP ARX - Model pomiaru ciśnienia KP1 w komorze paleniskowej", y_unit='-')

In [ ]:
#Displaying results LINEAR
display_test_results(get('test_results_KP1_LINEAR'), "MODEL 3 - Model pomiaru ciśnienia KP1 w komorze paleniskowej LINEAR ARX")

In [ ]:
display_train_val_test(get('train_results_KP1_LINEAR'), get('val_results_KP1_LINEAR'), get('test_results_KP1_LINEAR'), "MODEL 3 LINEAR ARX - Model pomiaru ciśnienia KP1 w komorze paleniskowej", y_unit='-')

## Metrics

In [ ]:
metrics = []
files = ["train_results_KP1_normalized_LSTM", "val_results_KP1_normalized_LSTM", "test_results_KP1_normalized_LSTM", "train_results_KP1_normalized_MLP", "val_results_KP1_normalized_MLP", "test_results_KP1_normalized_MLP", "train_results_KP1_normalized_LINEAR", "val_results_KP1_normalized_LINEAR", "test_results_KP1_normalized_LINEAR"]
for file in files:
  results = get(file).drop(columns=["Unnamed: 0"])
  me, mse, mae, rmse = calculate_metrics(results[results.columns[1]], results[results.columns[0]])#Actuals and then predicitons
  metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df['Model'] = ['LSTM train', 'LSTM val', 'LSTM test', 'MLP ARX train', 'MLP ARX val', 'MLP ARX test', 'LINEAR ARX train', 'LINEAR ARX val', 'LINEAR ARX test']
metrics_df.rename(columns={'Model': 'MODEL 3 - Model pomiaru ciśnienia KP1', 'ME': 'Mean_Error', 'MSE': 'Mean_Squared_Error', 
                           'MAE': 'Mean_Absolute_Error', 'RMSE': 'Root_Mean_Squared_Error'}, inplace=True)
metrics_df.to_csv("metrics/KP1_models_metrics_NEW.csv", index=False)

In [ ]:
metrics_df = pd.read_csv("metrics/KP1_models_metrics_NEW.csv")
metrics_df.iloc[6:, [0, 2, 4]].style.apply(highlight_min_green, subset=metrics_df.iloc[:, [ 2, 4]].columns)#['Mean_Squared_Error', 'Root_Mean_Squared_Error'], axis=0)

# MODEL 4 - Model pomiaru ciśnienia KO1 na wyjściu z kotła

In [ ]:
#Comparison
display_comparison(get('test_results_KO1_LSTM'), get('test_results_KO1_MLP'), get('test_results_KO1_LINEAR'), "MODEL 4 - Model pomiaru ciśnienia KO1 na wyjściu z kotła", y_unit='-')

In [ ]:
metrics = []

## LSTM

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  df_KO1 = file_data[col_KO1]
  
  X_train, y_train = data_to_X_y(df_KO1[col_KO1[0]], WINDOW_SIZE)
  var_train = df_KO1[col_KO1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data[col_KO1[0]], WINDOW_SIZE)
var_val = val_file_data[col_KO1[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

In [ ]:
#Model creation
model4 = create_model(18) #18 additional variables not including sequences

cp = ModelCheckpoint('model_KO1/model_KO1.keras', save_best_only=True)

model4.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model4.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model predictions
model4 = load_model('model_KO1/model_KO1.keras')

X_test, y_test = data_to_X_y(test_file_data[col_KO1[0]], WINDOW_SIZE)
var_test = test_file_data[col_KO1[1:]][WINDOW_SIZE-1:].to_numpy()
#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]

test = 0
window_test = X_test.shape[0]
# train = 0
# window_train = X_train.shape[0]
# val = 0
# window_val = X_val.shape[0]


# train_predictions = predict(model4, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_KO1 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(denorm(train_results_KO1, col_KO1[0]),'train_results_KO1')
# save(train_results_KO1,'train_results_KO1_normalized')

# val_predictions = predict(model4, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_KO1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(denorm(val_results_KO1, col_KO1[0]),'val_results_KO1')
# save(val_results_KO1,'val_results_KO1_normalized')

test_predictions = predict(model4, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test)
test_results_KO1 = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_KO1, col_KO1[0]), 'test_results_KO1')
save(test_results_KO1,'test_results_KO1_normalized')

In [ ]:
#Displaying results
display_test_results(get('test_results_KO1'), "MODEL 4 - Model pomiaru ciśnienia KO1 na wyjściu z kotła LSTM")
#display(get('train_results_KO1'), get('val_results_KO1'), get('test_results_KO1'),'all', "MODEL 4 - Model pomiaru ciśnienia KO1 na wyjściu z kotła LSTM")

## LSTM resampled data

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
   #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  df_KO1 = file_data[col_KO1]
  
  X_train, y_train = data_to_X_y(df_KO1[col_KO1[0]], WINDOW_SIZE)
  var_train = df_KO1[col_KO1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data_resampled[col_KO1[0]], WINDOW_SIZE)
var_val = val_file_data_resampled[col_KO1[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_KO1 = file_data[col_KO1]
  
  X_test, y_test = data_to_X_y(df_KO1[col_KO1[0]], WINDOW_SIZE)
  var_test = df_KO1[col_KO1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_test = min(X_test.shape[0],min(y_test.shape[0], var_test.shape[0]))
  X_test = X_test[:length_test] 
  y_test = y_test[:length_test] 
  var_test = var_test[:length_test] 
  
  test_data.append([X_test, y_test, var_test])

In [ ]:
#Model creation
model4 = create_model(18) #18 additional variables not including sequences

cp = ModelCheckpoint('model_KO1/model_KO1_LSTM.keras', save_best_only=True)

model4.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model4.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model4 = load_model('model_KO1/model_KO1_LSTM.keras')
train_results = []
for [X_train, y_train, var_train] in prepared_data:
  #We specify from which point we want to predict the next values for each dataset [s]
  #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict(model4, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)

train_results_KO1 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_KO1, col_KO1[0]), 'train_results_KO1_LSTM')
save(train_results_KO1,'train_results_KO1_normalized_LSTM')

val = 0
window_val = X_val.shape[0]
val_predictions = predict(model4, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val, y_val)
val_results_KO1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KO1, col_KO1[0]),'val_results_KO1_LSTM')
save(val_results_KO1,'val_results_KO1_normalized_LSTM')

test_results = []
for [X_test, y_test, var_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict(model4, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)

test_results_KO1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KO1, col_KO1[0]), 'test_results_KO1_LSTM')
save(test_results_KO1,'test_results_KO1_normalized_LSTM')

In [ ]:
#Displaying results
display_test_results(get('test_results_KO1_LSTM'), "MODEL 4 - Model pomiaru ciśnienia KO1 na wyjściu z kotła LSTM")

In [ ]:
display_train_val_test(get('train_results_KO1_LSTM'), get('val_results_KO1_LSTM'), get('test_results_KO1_LSTM'), "MODEL 4 LSTM - Model pomiaru ciśnienia KO1 na wyjściu z kotła", y_unit='-')

## MLP ARX and LINEAR ARX resampled data

In [ ]:
#Data preparation
#Przetrwarzamy dane tak, żeby można było przekazać dwa poprzednie kroki zmiennej jako parametry do naszego modelu

prepared_data = []
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  #Taking only the columns relevant for the model
  df_KP1 = file_data[col_KO1]
  
  # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_KP1['y_k_1'] = df_KP1[col_KO1[0]].shift(1)  # y[k-1]
  df_KP1['y_k_2'] = df_KP1[col_KO1[0]].shift(2)  # y[k-2]
  df_KP1 = df_KP1.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_train = df_KP1[col_KO1[1:] + ['y_k_1','y_k_2']]
  y_train = df_KP1[col_KO1[0]]
  
  prepared_data.append([X_train, y_train])
  
X_val = val_file_data_MLP[col_KO1[1:] + ['y_k_1','y_k_2']]
y_val = val_file_data_MLP[col_KO1[0]]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_KO1 = file_data[col_KO1]
  
    # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_KO1['y_k_1'] = df_KO1[col_KO1[0]].shift(1)  # y[k-1]
  df_KO1['y_k_2'] = df_KO1[col_KO1[0]].shift(2)  # y[k-2]
  df_KO1 = df_KO1.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_test = df_KO1[col_KO1[1:] + ['y_k_1','y_k_2']]
  y_test = df_KO1[col_KO1[0]]
  
  test_data.append([X_test, y_test])


In [ ]:
#Model creation
model4 = create_model_MLP(20) #20 variables (18+2y)

cp = ModelCheckpoint('model_KO1/model_KO1_MLP.keras', save_best_only=True)

model4.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model4.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model4 = load_model('model_KO1/model_KO1_MLP.keras')

train_results = []
for [X_train, y_train] in prepared_data:
  # #We specify from which point we want to predict the next values for each dataset [s]
  # #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict_MLP(model4, X_train, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)
  
train_results_KO1 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_KO1, col_KO1[0]), 'train_results_KO1_MLP')
save(train_results_KO1,'train_results_KO1_normalized_MLP')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_MLP(model4, X_val, window_val, y_val)
val_results_KO1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KO1, col_KO1[0]),'val_results_KO1_MLP')
save(val_results_KO1,'val_results_KO1_normalized_MLP')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_MLP(model4, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_KO1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KO1, col_KO1[0]), 'test_results_KO1_MLP')
save(test_results_KO1,'test_results_KO1_normalized_MLP')

In [ ]:
#LINEAR
#Model creation linear
model4 = LinearRegression()
#Model training linear
X_train = train_file_data_MLP[col_KO1[1:] + ['y_k_1','y_k_2']]
y_train = train_file_data_MLP[col_KO1[0]]

model4.fit(X_train, y_train)

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
train_predictions = predict_LINEAR(model4, X_train, window_train, y_train)
train_results_KO1 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_KO1, col_KO1[0]),'train_results_KO1_LINEAR')
save(train_results_KO1,'train_results_KO1_normalized_LINEAR')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_LINEAR(model4, X_val, window_val, y_val)
val_results_KO1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KO1, col_KO1[0]),'val_results_KO1_LINEAR')
save(val_results_KO1,'val_results_KO1_normalized_LINEAR')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_LINEAR(model4, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_KO1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KO1, col_KO1[0]), 'test_results_KO1_LINEAR')
save(test_results_KO1,'test_results_KO1_normalized_LINEAR')


In [ ]:
#Displaying results MLP
display_test_results(get('test_results_KO1_MLP'), "MODEL 4 - Model pomiaru ciśnienia KO1 na wyjściu z kotła MLP ARX")

In [ ]:
display_train_val_test(get('train_results_KO1_MLP'), get('val_results_KO1_MLP'), get('test_results_KO1_MLP'), "MODEL 4 MLP ARX - Model pomiaru ciśnienia KO1 na wyjściu z kotła", y_unit='-')

In [ ]:
#Displaying results LINEAR
display_test_results(get('test_results_KO1_LINEAR'), "MODEL 4 - Model pomiaru ciśnienia KO1 na wyjściu z kotła LINEAR ARX")

In [ ]:
display_train_val_test(get('train_results_KO1_LINEAR'), get('val_results_KO1_LINEAR'), get('test_results_KO1_LINEAR'), "MODEL 4 LINEAR ARX - Model pomiaru ciśnienia KO1 na wyjściu z kotła", y_unit='-')

## Metrics

In [ ]:
metrics = []
files = ["train_results_KO1_normalized_LSTM", "val_results_KO1_normalized_LSTM", "test_results_KO1_normalized_LSTM", "train_results_KO1_normalized_MLP", "val_results_KO1_normalized_MLP", "test_results_KO1_normalized_MLP", "train_results_KO1_normalized_LINEAR", "val_results_KO1_normalized_LINEAR", "test_results_KO1_normalized_LINEAR"]
for file in files:
  results = get(file).drop(columns=["Unnamed: 0"])
  me, mse, mae, rmse = calculate_metrics(results[results.columns[1]], results[results.columns[0]])#Actuals and then predicitons
  metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df['Model'] = ['LSTM train', 'LSTM val', 'LSTM test', 'MLP ARX train', 'MLP ARX val', 'MLP ARX test', 'LINEAR ARX train', 'LINEAR ARX val', 'LINEAR ARX test']
metrics_df.rename(columns={'Model': 'MODEL 4 - Model pomiaru ciśnienia KO1', 'ME': 'Mean_Error', 'MSE': 'Mean_Squared_Error', 
                           'MAE': 'Mean_Absolute_Error', 'RMSE': 'Root_Mean_Squared_Error'}, inplace=True)
metrics_df.to_csv("metrics/KO1_models_metrics_NEW.csv", index=False)

In [ ]:
metrics_df = pd.read_csv("metrics/KO1_models_metrics_NEW.csv")
metrics_df.iloc[6:, [0, 2, 4]].style.apply(highlight_min_green, subset=metrics_df.iloc[:, [ 2, 4]].columns)#['Mean_Squared_Error', 'Root_Mean_Squared_Error'], axis=0)

# MODEL 5 - Model pomiaru ciśnienia KO2 na wyjściu z kotła

In [ ]:
#Comparison
display_comparison(get('test_results_KO2_LSTM'), get('test_results_KO2_MLP'), get('test_results_KO2_LINEAR'), "MODEL 5 - Model pomiaru ciśnienia KO2 na wyjściu z kotła", y_unit='-')

In [ ]:
metrics = []

## LSTM

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  df_KO2 = file_data[col_KO2]
  
  X_train, y_train = data_to_X_y(df_KO2[col_KO2[0]], WINDOW_SIZE)
  var_train = df_KO2[col_KO2[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data[col_KO2[0]], WINDOW_SIZE)
var_val = val_file_data[col_KO2[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

In [ ]:
#Model creation
model5 = create_model(17) #17 additional variables not including sequences

cp = ModelCheckpoint('model_KO2/model_KO2.keras', save_best_only=True)

model5.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model5.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model predictions
model5 = load_model('model_KO2/model_KO2.keras')

X_test, y_test = data_to_X_y(test_file_data[col_KO2[0]], WINDOW_SIZE)
var_test = test_file_data[col_KO2[1:]][WINDOW_SIZE-1:].to_numpy()
#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]

test = 0
window_test = X_test.shape[0]
# train = 0
# window_train = X_train.shape[0]
# val = 0
# window_val = X_val.shape[0]


# train_predictions = predict(model5, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_KO2 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(denorm(train_results_KO2, col_KO2[0]),'train_results_KO2')
# save(train_results_KO2,'train_results_KO2_normalized')

# val_predictions = predict(model5, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_KO2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(denorm(val_results_KO2, col_KO2[0]),'val_results_KO2')
# save(val_results_KO2,'val_results_KO2_normalized')

test_predictions = predict(model5, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test)
test_results_KO2 = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_KO2, col_KO2[0]), 'test_results_KO2')
save(test_results_KO2,'test_results_KO2_normalized')

In [ ]:
#Displaying results
display_test_results(get('test_results_KO2'), "MODEL 5 - Model pomiaru ciśnienia KO2 na wyjściu z kotła LSTM")
#display(get('train_results_KO2'), get('val_results_KO2'), get('test_results_KO2'),'all', "MODEL 5 - Model pomiaru ciśnienia KO2 na wyjściu z kotła LSTM")

## LSTM resampled data

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  df_KO2 = file_data[col_KO2]
  
  X_train, y_train = data_to_X_y(df_KO2[col_KO2[0]], WINDOW_SIZE)
  var_train = df_KO2[col_KO2[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data_resampled[col_KO2[0]], WINDOW_SIZE)
var_val = val_file_data_resampled[col_KO2[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_KO2 = file_data[col_KO2]
  
  X_test, y_test = data_to_X_y(df_KO2[col_KO2[0]], WINDOW_SIZE)
  var_test = df_KO2[col_KO2[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_test = min(X_test.shape[0],min(y_test.shape[0], var_test.shape[0]))
  X_test = X_test[:length_test] 
  y_test = y_test[:length_test] 
  var_test = var_test[:length_test] 
  
  test_data.append([X_test, y_test, var_test])

In [ ]:
#Model creation
model5 = create_model(17) #17 additional variables not including sequences

cp = ModelCheckpoint('model_KO2/model_KO2_LSTM.keras', save_best_only=True)

model5.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model5.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model5 = load_model('model_KO2/model_KO2_LSTM.keras')
train_results = []
for [X_train, y_train, var_train] in prepared_data:
  #We specify from which point we want to predict the next values for each dataset [s]
  #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict(model5, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)

train_results_KO2 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_KO2, col_KO2[0]), 'train_results_KO2_LSTM')
save(train_results_KO2,'train_results_KO2_normalized_LSTM')

val = 0
window_val = X_val.shape[0]
val_predictions = predict(model5, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val, y_val)
val_results_KO2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KO2, col_KO2[0]),'val_results_KO2_LSTM')
save(val_results_KO2,'val_results_KO2_normalized_LSTM')

test_results = []
for [X_test, y_test, var_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict(model5, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)

test_results_KO2 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KO2, col_KO2[0]), 'test_results_KO2_LSTM')
save(test_results_KO2,'test_results_KO2_normalized_LSTM')

In [ ]:
#Displaying results
display_test_results(get('test_results_KO2_LSTM'), "MODEL 5 - Model pomiaru ciśnienia KO2 na wyjściu z kotła LSTM")

In [ ]:
display_train_val_test(get('train_results_KO2_LSTM'), get('val_results_KO2_LSTM'), get('test_results_KO2_LSTM'), "MODEL 5 LSTM - Model pomiaru ciśnienia KO2 na wyjściu z kotła", y_unit='-')

## MLP ARX and LINEAR ARX resampled data

In [ ]:
#Data preparation
#Przetrwarzamy dane tak, żeby można było przekazać dwa poprzednie kroki zmiennej jako parametry do naszego modelu

prepared_data = []
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  #Taking only the columns relevant for the model
  df_KO2 = file_data[col_KO2]
  
  # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_KO2['y_k_1'] = df_KO2[col_KO2[0]].shift(1)  # y[k-1]
  df_KO2['y_k_2'] = df_KO2[col_KO2[0]].shift(2)  # y[k-2]
  df_KO2 = df_KO2.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_train = df_KO2[col_KO2[1:] + ['y_k_1','y_k_2']]
  y_train = df_KO2[col_KO2[0]]
  
  prepared_data.append([X_train, y_train])
  
X_val = val_file_data_MLP[col_KO2[1:] + ['y_k_1','y_k_2']]
y_val = val_file_data_MLP[col_KO2[0]]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_KO2 = file_data[col_KO2]
  
    # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_KO2['y_k_1'] = df_KO2[col_KO2[0]].shift(1)  # y[k-1]
  df_KO2['y_k_2'] = df_KO2[col_KO2[0]].shift(2)  # y[k-2]
  df_KO2 = df_KO2.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_test = df_KO2[col_KO2[1:] + ['y_k_1','y_k_2']]
  y_test = df_KO2[col_KO2[0]]
  
  test_data.append([X_test, y_test])


In [ ]:
#Model creation
model5 = create_model_MLP(19) #19 variables (17+2y)

cp = ModelCheckpoint('model_KO2/model_KO2_MLP.keras', save_best_only=True)

model5.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model5.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model5 = load_model('model_KO2/model_KO2_MLP.keras')

train_results = []
for [X_train, y_train] in prepared_data:
  # #We specify from which point we want to predict the next values for each dataset [s]
  # #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict_MLP(model5, X_train, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)
  
train_results_KO2 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_KO2, col_KO2[0]), 'train_results_KO2_MLP')
save(train_results_KO2,'train_results_KO2_normalized_MLP')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_MLP(model5, X_val, window_val, y_val)
val_results_KO2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KO2, col_KO2[0]),'val_results_KO2_MLP')
save(val_results_KO2,'val_results_KO2_normalized_MLP')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_MLP(model5, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_KO2 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KO2, col_KO2[0]), 'test_results_KO2_MLP')
save(test_results_KO2,'test_results_KO2_normalized_MLP')

In [ ]:
#LINEAR
#Model creation linear
model5 = LinearRegression()
#Model training linear
X_train = train_file_data_MLP[col_KO2[1:] + ['y_k_1','y_k_2']]
y_train = train_file_data_MLP[col_KO2[0]]

model5.fit(X_train, y_train)

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
train_predictions = predict_LINEAR(model5, X_train, window_train, y_train)
train_results_KO2 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_KO2, col_KO2[0]),'train_results_KO2_LINEAR')
save(train_results_KO2,'train_results_KO2_normalized_LINEAR')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_LINEAR(model5, X_val, window_val, y_val)
val_results_KO2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KO2, col_KO2[0]),'val_results_KO2_LINEAR')
save(val_results_KO2,'val_results_KO2_normalized_LINEAR')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_LINEAR(model5, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_KO2 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KO2, col_KO2[0]), 'test_results_KO2_LINEAR')
save(test_results_KO2,'test_results_KO2_normalized_LINEAR')


In [ ]:
#Displaying results MLP
display_test_results(get('test_results_KO2_MLP'), "MODEL 5 - Model pomiaru ciśnienia KO2 na wyjściu z kotła MLP ARX")

In [ ]:
display_train_val_test(get('train_results_KO2_MLP'), get('val_results_KO2_MLP'), get('test_results_KO2_MLP'), "MODEL 5 MLP ARX - Model pomiaru ciśnienia KO2 na wyjściu z kotła", y_unit='-')

In [ ]:
#Displaying results LINEAR
display_test_results(get('test_results_KO2_LINEAR'), "MODEL 5 - Model pomiaru ciśnienia KO2 na wyjściu z kotła LINEAR ARX")

In [ ]:
display_train_val_test(get('train_results_KO2_LINEAR'), get('val_results_KO2_LINEAR'), get('test_results_KO2_LINEAR'), "MODEL 5 LINEAR ARX - Model pomiaru ciśnienia KO2 na wyjściu z kotła", y_unit='-')

## Metrics

In [ ]:
metrics = []
files = ["train_results_KO2_normalized_LSTM", "val_results_KO2_normalized_LSTM", "test_results_KO2_normalized_LSTM", "train_results_KO2_normalized_MLP", "val_results_KO2_normalized_MLP", "test_results_KO2_normalized_MLP", "train_results_KO2_normalized_LINEAR", "val_results_KO2_normalized_LINEAR", "test_results_KO2_normalized_LINEAR"]
for file in files:
  results = get(file).drop(columns=["Unnamed: 0"])
  me, mse, mae, rmse = calculate_metrics(results[results.columns[1]], results[results.columns[0]])#Actuals and then predicitons
  metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df['Model'] = ['LSTM train', 'LSTM val', 'LSTM test', 'MLP ARX train', 'MLP ARX val', 'MLP ARX test', 'LINEAR ARX train', 'LINEAR ARX val', 'LINEAR ARX test']
metrics_df.rename(columns={'Model': 'MODEL 5 - Model pomiaru ciśnienia KO2', 'ME': 'Mean_Error', 'MSE': 'Mean_Squared_Error', 
                           'MAE': 'Mean_Absolute_Error', 'RMSE': 'Root_Mean_Squared_Error'}, inplace=True)
metrics_df.to_csv("metrics/KO2_models_metrics_NEW.csv", index=False)

In [ ]:
metrics_df = pd.read_csv("metrics/KO2_models_metrics_NEW.csv")
metrics_df.iloc[6:, [0, 2, 4]].style.apply(highlight_min_green, subset=metrics_df.iloc[:, [ 2, 4]].columns)#['Mean_Squared_Error', 'Root_Mean_Squared_Error'], axis=0)

# MODEL 6 - Model pomiaru ciśnienia spalin KS1

In [ ]:
#Comparison
display_comparison(get('test_results_KS1_LSTM'), get('test_results_KS1_MLP'), get('test_results_KS1_LINEAR'), "MODEL 6 - Model pomiaru ciśnienia spalin KS1", y_unit='-')

In [ ]:
metrics = []

## LSTM

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  df_KS1 = file_data[col_KS1]
  
  X_train, y_train = data_to_X_y(df_KS1[col_KS1[0]], WINDOW_SIZE)
  var_train = df_KS1[col_KS1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data[col_KS1[0]], WINDOW_SIZE)
var_val = val_file_data[col_KS1[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

In [ ]:
#Model creation
model6 = create_model(4) #4 additional variables not including sequences

cp = ModelCheckpoint('model_KS1/model_KS1.keras', save_best_only=True)

model6.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model6.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model predictions
model6 = load_model('model_KS1/model_KS1.keras')

X_test, y_test = data_to_X_y(test_file_data[col_KS1[0]], WINDOW_SIZE)
var_test = test_file_data[col_KS1[1:]][WINDOW_SIZE-1:].to_numpy()
#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]

test = 0
window_test = X_test.shape[0]
# train = 0
# window_train = X_train.shape[0]
# val = 0
# window_val = X_val.shape[0]


# train_predictions = predict(model6, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_KS1 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(denorm(train_results_KS1, col_KS1[0]),'train_results_KS1')
# save(train_results_KS1,'train_results_KS1_normalized')

# val_predictions = predict(model6, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_KS1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(denorm(val_results_KS1, col_KS1[0]),'val_results_KS1')
# save(val_results_KS1,'val_results_KS1_normalized')

test_predictions = predict(model6, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test)
test_results_KS1 = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_KS1, col_KS1[0]), 'test_results_KS1')
save(test_results_KS1,'test_results_KS1_normalized')

In [ ]:
#Displaying results
display_test_results(get('test_results_KS1'), "MODEL 6 - Model pomiaru ciśnienia spalin KS1 LSTM")
#display(get('train_results_KS1'), get('val_results_KS1'), get('test_results_KS1'),'all', "MODEL 6 - Model pomiaru ciśnienia spalin KS1 LSTM")

## LSTM resampled data

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
   #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  df_KS1 = file_data[col_KS1]
  
  X_train, y_train = data_to_X_y(df_KS1[col_KS1[0]], WINDOW_SIZE)
  var_train = df_KS1[col_KS1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data_resampled[col_KS1[0]], WINDOW_SIZE)
var_val = val_file_data_resampled[col_KS1[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_KS1 = file_data[col_KS1]
  
  X_test, y_test = data_to_X_y(df_KS1[col_KS1[0]], WINDOW_SIZE)
  var_test = df_KS1[col_KS1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_test = min(X_test.shape[0],min(y_test.shape[0], var_test.shape[0]))
  X_test = X_test[:length_test] 
  y_test = y_test[:length_test] 
  var_test = var_test[:length_test] 
  
  test_data.append([X_test, y_test, var_test])

In [ ]:
#Model creation
model6 = create_model(4) #4 additional variables not including sequences

cp = ModelCheckpoint('model_KS1/model_KS1_LSTM.keras', save_best_only=True)

model6.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model6.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model6 = load_model('model_KS1/model_KS1_LSTM.keras')
train_results = []
for [X_train, y_train, var_train] in prepared_data:
  #We specify from which point we want to predict the next values for each dataset [s]
  #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict(model6, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)

train_results_KS1 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_KS1, col_KS1[0]), 'train_results_KS1_LSTM')
save(train_results_KS1,'train_results_KS1_normalized_LSTM')

val = 0
window_val = X_val.shape[0]
val_predictions = predict(model6, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val, y_val)
val_results_KS1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KS1, col_KS1[0]),'val_results_KS1_LSTM')
save(val_results_KS1,'val_results_KS1_normalized_LSTM')

test_results = []
for [X_test, y_test, var_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict(model6, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)

test_results_KS1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KS1, col_KS1[0]), 'test_results_KS1_LSTM')
save(test_results_KS1,'test_results_KS1_normalized_LSTM')

In [ ]:
#Displaying results
display_test_results(get('test_results_KS1_LSTM'), "MODEL 6 - Model pomiaru ciśnienia spalin KS1 LSTM")

In [ ]:
display_train_val_test(get('train_results_KS1_LSTM'), get('val_results_KS1_LSTM'), get('test_results_KS1_LSTM'), "MODEL 6 LSTM - Model pomiaru ciśnienia spalin KS1", y_unit='-')

## MLP ARX and LINEAR ARX resampled data

In [ ]:
#Data preparation
#Przetrwarzamy dane tak, żeby można było przekazać dwa poprzednie kroki zmiennej jako parametry do naszego modelu

prepared_data = []
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  #Taking only the columns relevant for the model
  df_KS1 = file_data[col_KS1]
  
  # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_KS1['y_k_1'] = df_KS1[col_KS1[0]].shift(1)  # y[k-1]
  df_KS1['y_k_2'] = df_KS1[col_KS1[0]].shift(2)  # y[k-2]
  df_KS1 = df_KS1.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_train = df_KS1[col_KS1[1:] + ['y_k_1','y_k_2']]
  y_train = df_KS1[col_KS1[0]]
  
  prepared_data.append([X_train, y_train])
  
X_val = val_file_data_MLP[col_KS1[1:] + ['y_k_1','y_k_2']]
y_val = val_file_data_MLP[col_KS1[0]]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_KS1 = file_data[col_KS1]
  
    # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_KS1['y_k_1'] = df_KS1[col_KS1[0]].shift(1)  # y[k-1]
  df_KS1['y_k_2'] = df_KS1[col_KS1[0]].shift(2)  # y[k-2]
  df_KS1 = df_KS1.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_test = df_KS1[col_KS1[1:] + ['y_k_1','y_k_2']]
  y_test = df_KS1[col_KS1[0]]
  
  test_data.append([X_test, y_test])


In [ ]:
#Model creation
model6 = create_model_MLP(6) #6 variables (4+2y)

cp = ModelCheckpoint('model_KS1/model_KS1_MLP.keras', save_best_only=True)

model6.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model6.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model6 = load_model('model_KS1/model_KS1_MLP.keras')

train_results = []
for [X_train, y_train] in prepared_data:
  # #We specify from which point we want to predict the next values for each dataset [s]
  # #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict_MLP(model6, X_train, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)
  
train_results_KS1 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_KS1, col_KS1[0]), 'train_results_KS1_MLP')
save(train_results_KS1,'train_results_KS1_normalized_MLP')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_MLP(model6, X_val, window_val, y_val)
val_results_KS1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KS1, col_KS1[0]),'val_results_KS1_MLP')
save(val_results_KS1,'val_results_KS1_normalized_MLP')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_MLP(model6, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_KS1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KS1, col_KS1[0]), 'test_results_KS1_MLP')
save(test_results_KS1,'test_results_KS1_normalized_MLP')

In [ ]:
#LINEAR
#Model creation linear
model6 = LinearRegression()
#Model training linear
X_train = train_file_data_MLP[col_KS1[1:] + ['y_k_1','y_k_2']]
y_train = train_file_data_MLP[col_KS1[0]]

model6.fit(X_train, y_train)

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
train_predictions = predict_LINEAR(model6, X_train, window_train, y_train)
train_results_KS1 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_KS1, col_KS1[0]),'train_results_KS1_LINEAR')
save(train_results_KS1,'train_results_KS1_normalized_LINEAR')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_LINEAR(model6, X_val, window_val, y_val)
val_results_KS1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KS1, col_KS1[0]),'val_results_KS1_LINEAR')
save(val_results_KS1,'val_results_KS1_normalized_LINEAR')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_LINEAR(model6, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_KS1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KS1, col_KS1[0]), 'test_results_KS1_LINEAR')
save(test_results_KS1,'test_results_KS1_normalized_LINEAR')


In [ ]:
#Displaying results MLP
display_test_results(get('test_results_KS1_MLP'), "MODEL 6 - Model pomiaru ciśnienia spalin KS1 MLP ARX")

In [ ]:
display_train_val_test(get('train_results_KS1_MLP'), get('val_results_KS1_MLP'), get('test_results_KS1_MLP'), "MODEL 6 MLP ARX - Model pomiaru ciśnienia spalin KS1", y_unit='-')

In [ ]:
#Displaying results LINEAR
display_test_results(get('test_results_KS1_LINEAR'), "MODEL 6 - Model pomiaru ciśnienia spalin KS1 LINEAR ARX")

In [ ]:
display_train_val_test(get('train_results_KS1_LINEAR'), get('val_results_KS1_LINEAR'), get('test_results_KS1_LINEAR'), "MODEL 6 LINEAR ARX - Model pomiaru ciśnienia spalin KS1", y_unit='-')

## Metrics

In [ ]:
metrics = []
files = ["train_results_KS1_normalized_LSTM", "val_results_KS1_normalized_LSTM", "test_results_KS1_normalized_LSTM", "train_results_KS1_normalized_MLP", "val_results_KS1_normalized_MLP", "test_results_KS1_normalized_MLP", "train_results_KS1_normalized_LINEAR", "val_results_KS1_normalized_LINEAR", "test_results_KS1_normalized_LINEAR"]
for file in files:
  results = get(file).drop(columns=["Unnamed: 0"])
  me, mse, mae, rmse = calculate_metrics(results[results.columns[1]], results[results.columns[0]])#Actuals and then predicitons
  metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df['Model'] = ['LSTM train', 'LSTM val', 'LSTM test', 'MLP ARX train', 'MLP ARX val', 'MLP ARX test', 'LINEAR ARX train', 'LINEAR ARX val', 'LINEAR ARX test']
metrics_df.rename(columns={'Model': 'MODEL 6 - Model pomiaru ciśnienia spalin KS1', 'ME': 'Mean_Error', 'MSE': 'Mean_Squared_Error', 
                           'MAE': 'Mean_Absolute_Error', 'RMSE': 'Root_Mean_Squared_Error'}, inplace=True)
metrics_df.to_csv("metrics/KS1_models_metrics_NEW.csv", index=False)

In [ ]:
metrics_df = pd.read_csv("metrics/KS1_models_metrics_NEW.csv")
metrics_df.iloc[6:, [0, 2, 4]].style.apply(highlight_min_green, subset=metrics_df.iloc[:, [ 2, 4]].columns)#['Mean_Squared_Error', 'Root_Mean_Squared_Error'], axis=0)

# MODEL 7 - Model pomiaru ciśnienia spalin KS2

In [ ]:
#Comparison
display_comparison(get('test_results_KS2_LSTM'), get('test_results_KS2_MLP'), get('test_results_KS2_LINEAR'), "MODEL 7 - Model pomiaru ciśnienia spalin KS2", y_unit='-')

In [ ]:
metrics = []

## LSTM

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  df_KS2 = file_data[col_KS2]
  
  X_train, y_train = data_to_X_y(df_KS2[col_KS2[0]], WINDOW_SIZE)
  var_train = df_KS2[col_KS2[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data[col_KS2[0]], WINDOW_SIZE)
var_val = val_file_data[col_KS2[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

In [ ]:
#Model creation
model7 = create_model(5) #5 additional variables not including sequences

cp = ModelCheckpoint('model_KS2/model_KS2.keras', save_best_only=True)

model7.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model7.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model predictions
model7 = load_model('model_KS2/model_KS2.keras')

X_test, y_test = data_to_X_y(test_file_data[col_KS2[0]], WINDOW_SIZE)
var_test = test_file_data[col_KS2[1:]][WINDOW_SIZE-1:].to_numpy()
#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]

test = 0
window_test = X_test.shape[0]
# train = 0
# window_train = X_train.shape[0]
# val = 0
# window_val = X_val.shape[0]


# train_predictions = predict(model7, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_KS2 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(denorm(train_results_KS2, col_KS2[0]),'train_results_KS2')
# save(train_results_KS2,'train_results_KS2_normalized')

# val_predictions = predict(model7, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_KS2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(denorm(val_results_KS2, col_KS2[0]),'val_results_KS2')
# save(val_results_KS2,'val_results_KS2_normalized')

test_predictions = predict(model7, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test)
test_results_KS2 = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_KS2, col_KS2[0]), 'test_results_KS2')
save(test_results_KS2,'test_results_KS2_normalized')

In [ ]:
#Displaying results
display_test_results(get('test_results_KS2'), "MODEL 7 - Model pomiaru ciśnienia spalin KS2 LSTM")
#display(get('train_results_KS2'), get('val_results_KS2'), get('test_results_KS2'),'all', "MODEL 7 - Model pomiaru ciśnienia spalin KS2 LSTM")

## LSTM resampled data

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
   #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  df_KS2 = file_data[col_KS2]
  
  X_train, y_train = data_to_X_y(df_KS2[col_KS2[0]], WINDOW_SIZE)
  var_train = df_KS2[col_KS2[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data_resampled[col_KS2[0]], WINDOW_SIZE)
var_val = val_file_data_resampled[col_KS2[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_KS2 = file_data[col_KS2]
  
  X_test, y_test = data_to_X_y(df_KS2[col_KS2[0]], WINDOW_SIZE)
  var_test = df_KS2[col_KS2[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_test = min(X_test.shape[0],min(y_test.shape[0], var_test.shape[0]))
  X_test = X_test[:length_test] 
  y_test = y_test[:length_test] 
  var_test = var_test[:length_test] 
  
  test_data.append([X_test, y_test, var_test])

In [ ]:
#Model creation
model7 = create_model(5) #5 additional variables not including sequences

cp = ModelCheckpoint('model_KS2/model_KS2_LSTM.keras', save_best_only=True)

model7.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model7.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model7 = load_model('model_KS2/model_KS2_LSTM.keras')
train_results = []
for [X_train, y_train, var_train] in prepared_data:
  #We specify from which point we want to predict the next values for each dataset [s]
  #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict(model7, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)

train_results_KS2 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_KS2, col_KS2[0]), 'train_results_KS2_LSTM')
save(train_results_KS2,'train_results_KS2_normalized_LSTM')

val = 0
window_val = X_val.shape[0]
val_predictions = predict(model7, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val, y_val)
val_results_KS2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KS2, col_KS2[0]),'val_results_KS2_LSTM')
save(val_results_KS2,'val_results_KS2_normalized_LSTM')

test_results = []
for [X_test, y_test, var_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict(model7, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)

test_results_KS2 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KS2, col_KS2[0]), 'test_results_KS2_LSTM')
save(test_results_KS2,'test_results_KS2_normalized_LSTM')

In [ ]:
#Displaying results
display_test_results(get('test_results_KS2_LSTM'), "MODEL 7 - Model pomiaru ciśnienia spalin KS2 LSTM")

In [ ]:
display_train_val_test(get('train_results_KS2_LSTM'), get('val_results_KS2_LSTM'), get('test_results_KS2_LSTM'), "MODEL 7 LSTM - Model pomiaru ciśnienia spalin KS2", y_unit='-')

## MLP ARX and LINEAR ARX resampled data

In [ ]:
#Data preparation
#Przetrwarzamy dane tak, żeby można było przekazać dwa poprzednie kroki zmiennej jako parametry do naszego modelu

prepared_data = []
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  #Taking only the columns relevant for the model
  df_KS2 = file_data[col_KS2]
  
  # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_KS2['y_k_1'] = df_KS2[col_KS2[0]].shift(1)  # y[k-1]
  df_KS2['y_k_2'] = df_KS2[col_KS2[0]].shift(2)  # y[k-2]
  df_KS2 = df_KS2.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_train = df_KS2[col_KS2[1:] + ['y_k_1','y_k_2']]
  y_train = df_KS2[col_KS2[0]]
  
  prepared_data.append([X_train, y_train])
  
X_val = val_file_data_MLP[col_KS2[1:] + ['y_k_1','y_k_2']]
y_val = val_file_data_MLP[col_KS2[0]]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_KS2 = file_data[col_KS2]
  
    # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_KS2['y_k_1'] = df_KS2[col_KS2[0]].shift(1)  # y[k-1]
  df_KS2['y_k_2'] = df_KS2[col_KS2[0]].shift(2)  # y[k-2]
  df_KS2 = df_KS2.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_test = df_KS2[col_KS2[1:] + ['y_k_1','y_k_2']]
  y_test = df_KS2[col_KS2[0]]
  
  test_data.append([X_test, y_test])

In [ ]:
#Model creation
model7 = create_model_MLP(7) #7 variables (5+2y)

cp = ModelCheckpoint('model_KS2/model_KS2_MLP.keras', save_best_only=True)

model7.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model7.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model7 = load_model('model_KS2/model_KS2_MLP.keras')

train_results = []
for [X_train, y_train] in prepared_data:
  # #We specify from which point we want to predict the next values for each dataset [s]
  # #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict_MLP(model7, X_train, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)
  
train_results_KS2 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_KS2, col_KS2[0]), 'train_results_KS2_MLP')
save(train_results_KS2,'train_results_KS2_normalized_MLP')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_MLP(model7, X_val, window_val, y_val)
val_results_KS2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KS2, col_KS2[0]),'val_results_KS2_MLP')
save(val_results_KS2,'val_results_KS2_normalized_MLP')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_MLP(model7, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_KS2 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KS2, col_KS2[0]), 'test_results_KS2_MLP')
save(test_results_KS2,'test_results_KS2_normalized_MLP')

In [ ]:
#LINEAR
#Model creation linear
model7 = LinearRegression()
#Model training linear
X_train = train_file_data_MLP[col_KS2[1:] + ['y_k_1','y_k_2']]
y_train = train_file_data_MLP[col_KS2[0]]

model7.fit(X_train, y_train)

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
train_predictions = predict_LINEAR(model7, X_train, window_train, y_train)
train_results_KS2 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_KS2, col_KS2[0]),'train_results_KS2_LINEAR')
save(train_results_KS2,'train_results_KS2_normalized_LINEAR')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_LINEAR(model7, X_val, window_val, y_val)
val_results_KS2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_KS2, col_KS2[0]),'val_results_KS2_LINEAR')
save(val_results_KS2,'val_results_KS2_normalized_LINEAR')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_LINEAR(model7, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_KS2 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_KS2, col_KS2[0]), 'test_results_KS2_LINEAR')
save(test_results_KS2,'test_results_KS2_normalized_LINEAR')


In [ ]:
#Displaying results MLP
display_test_results(get('test_results_KS2_MLP'), "MODEL 7 - Model pomiaru ciśnienia spalin KS2 MLP ARX")

In [ ]:
display_train_val_test(get('train_results_KS2_MLP'), get('val_results_KS2_MLP'), get('test_results_KS2_MLP'), "MODEL 7 MLP ARX - Model pomiaru ciśnienia spalin KS2", y_unit='-')

In [ ]:
#Displaying results LINEAR
display_test_results(get('test_results_KS2_LINEAR'), "MODEL 7 - Model pomiaru ciśnienia spalin KS2 LINEAR ARX")

In [ ]:
display_train_val_test(get('train_results_KS2_LINEAR'), get('val_results_KS2_LINEAR'), get('test_results_KS2_LINEAR'), "MODEL 7 LINEAR ARX - Model pomiaru ciśnienia spalin KS2", y_unit='-')

## Metrics

In [ ]:
metrics = []
files = ["train_results_KS2_normalized_LSTM", "val_results_KS2_normalized_LSTM", "test_results_KS2_normalized_LSTM", "train_results_KS2_normalized_MLP", "val_results_KS2_normalized_MLP", "test_results_KS2_normalized_MLP", "train_results_KS2_normalized_LINEAR", "val_results_KS2_normalized_LINEAR", "test_results_KS2_normalized_LINEAR"]
for file in files:
  results = get(file).drop(columns=["Unnamed: 0"])
  me, mse, mae, rmse = calculate_metrics(results[results.columns[1]], results[results.columns[0]])#Actuals and then predicitons
  metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df['Model'] = ['LSTM train', 'LSTM val', 'LSTM test', 'MLP ARX train', 'MLP ARX val', 'MLP ARX test', 'LINEAR ARX train', 'LINEAR ARX val', 'LINEAR ARX test']
metrics_df.rename(columns={'Model': 'MODEL 7 - Model pomiaru ciśnienia spalin KS2', 'ME': 'Mean_Error', 'MSE': 'Mean_Squared_Error', 
                           'MAE': 'Mean_Absolute_Error', 'RMSE': 'Root_Mean_Squared_Error'}, inplace=True)
metrics_df.to_csv("metrics/KS2_models_metrics_NEW.csv", index=False)

In [ ]:
metrics_df = pd.read_csv("metrics/KS2_models_metrics_NEW.csv")
metrics_df.iloc[6:, [0, 2, 4]].style.apply(highlight_min_green, subset=metrics_df.iloc[:, [ 2, 4]].columns)#['Mean_Squared_Error', 'Root_Mean_Squared_Error'], axis=0)

# MODEL 8 - Model pomiaru zawartości tlenu w spalinach O1

In [ ]:
#Comparison
display_comparison(get('test_results_O1_LSTM'), get('test_results_O1_MLP'), get('test_results_O1_LINEAR'), "MODEL 8 - Model pomiaru zawartości tlenu w spalinach O1", y_unit = '%')

In [ ]:
metrics = []

## LSTM

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  df_O1 = file_data[col_O1]
  
  X_train, y_train = data_to_X_y(df_O1[col_O1[0]], WINDOW_SIZE)
  var_train = df_O1[col_O1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data[col_O1[0]], WINDOW_SIZE)
var_val = val_file_data[col_O1[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

In [ ]:
#Model creation
model8 = create_model(16) #16 additional variables not including sequences

cp = ModelCheckpoint('model_O1/model_O1.keras', save_best_only=True)

model8.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model8.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model predictions
model8 = load_model('model_O1/model_O1.keras')

X_test, y_test = data_to_X_y(test_file_data[col_O1[0]], WINDOW_SIZE)
var_test = test_file_data[col_O1[1:]][WINDOW_SIZE-1:].to_numpy()
#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]

test = 0
window_test = X_test.shape[0]
# train = 0
# window_train = X_train.shape[0]
# val = 0
# window_val = X_val.shape[0]


# train_predictions = predict(model8, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_01 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(denorm(train_results_01, col_O1[0]),'train_results_01')
# save(train_results_01,'train_results_O1_normalized')

# val_predictions = predict(model8, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_O1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(denorm(val_results_O1, col_O1[0]),'val_results_O1')
# save(val_results_O1,'val_results_O1_normalized')

test_predictions = predict(model8, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test)
test_results_O1 = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_O1, col_O1[0]), 'test_results_O1')
save(test_results_O1,'test_results_O1_normalized')

In [ ]:
#Displaying results
display_test_results(get('test_results_O1'), "MODEL 8 - Model pomiaru zawartości tlenu w spalinach O1 LSTM")
#display(get('train_results_O1'), get('val_results_O1'), get('test_results_O1'),'all', "MODEL 8 - Model pomiaru zawartości tlenu w spalinach O1 LSTM")

## LSTM resampled data

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
   #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  df_O1 = file_data[col_O1]
  
  X_train, y_train = data_to_X_y(df_O1[col_O1[0]], WINDOW_SIZE)
  var_train = df_O1[col_O1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data_resampled[col_O1[0]], WINDOW_SIZE)
var_val = val_file_data_resampled[col_O1[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_O1 = file_data[col_O1]
  
  X_test, y_test = data_to_X_y(df_O1[col_O1[0]], WINDOW_SIZE)
  var_test = df_O1[col_O1[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_test = min(X_test.shape[0],min(y_test.shape[0], var_test.shape[0]))
  X_test = X_test[:length_test] 
  y_test = y_test[:length_test] 
  var_test = var_test[:length_test] 
  
  test_data.append([X_test, y_test, var_test])

In [ ]:
#Model creation
model8 = create_model(16) #16 additional variables not including sequences

cp = ModelCheckpoint('model_O1/model_O1_LSTM.keras', save_best_only=True)

model8.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model8.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model8 = load_model('model_O1/model_O1_LSTM.keras')
train_results = []
for [X_train, y_train, var_train] in prepared_data:
  #We specify from which point we want to predict the next values for each dataset [s]
  #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict(model8, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)

train_results_O1 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_O1, col_O1[0]), 'train_results_O1_LSTM')
save(train_results_O1,'train_results_O1_normalized_LSTM')

val = 0
window_val = X_val.shape[0]
val_predictions = predict(model8, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val, y_val)
val_results_O1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_O1, col_O1[0]),'val_results_O1_LSTM')
save(val_results_O1,'val_results_O1_normalized_LSTM')

test_results = []
for [X_test, y_test, var_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict(model8, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)

test_results_O1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_O1, col_O1[0]), 'test_results_O1_LSTM')
save(test_results_O1,'test_results_O1_normalized_LSTM')

In [ ]:
#Displaying results
display_test_results(get('test_results_O1_LSTM'), "MODEL 8 - Model pomiaru zawartości tlenu w spalinach O1 LSTM")

In [ ]:
display_train_val_test(get('train_results_O1_LSTM'), get('val_results_O1_LSTM'), get('test_results_O1_LSTM'), "MODEL 8 LSTM - Model pomiaru zawartości tlenu w spalinach O1", y_unit='%')

## MLP ARX and LINEAR ARX resampled data

In [ ]:
#Data preparation
#Przetrwarzamy dane tak, żeby można było przekazać dwa poprzednie kroki zmiennej jako parametry do naszego modelu

prepared_data = []
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  #Taking only the columns relevant for the model
  df_O1 = file_data[col_O1]
  
  # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_O1['y_k_1'] = df_O1[col_O1[0]].shift(1)  # y[k-1]
  df_O1['y_k_2'] = df_O1[col_O1[0]].shift(2)  # y[k-2]
  df_O1 = df_O1.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_train = df_O1[col_O1[1:] + ['y_k_1','y_k_2']]
  y_train = df_O1[col_O1[0]]
  
  prepared_data.append([X_train, y_train])
  
X_val = val_file_data_MLP[col_O1[1:] + ['y_k_1','y_k_2']]
y_val = val_file_data_MLP[col_O1[0]]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_O1 = file_data[col_O1]
  
    # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_O1['y_k_1'] = df_O1[col_O1[0]].shift(1)  # y[k-1]
  df_O1['y_k_2'] = df_O1[col_O1[0]].shift(2)  # y[k-2]
  df_O1 = df_O1.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_test = df_O1[col_O1[1:] + ['y_k_1','y_k_2']]
  y_test = df_O1[col_O1[0]]
  
  test_data.append([X_test, y_test])


In [ ]:
#Model creation
model8 = create_model_MLP(18) #18 variables (16+2y)

cp = ModelCheckpoint('model_O1/model_O1_MLP.keras', save_best_only=True)

model8.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model8.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model8 = load_model('model_O1/model_O1_MLP.keras')

train_results = []
for [X_train, y_train] in prepared_data:
  # #We specify from which point we want to predict the next values for each dataset [s]
  # #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict_MLP(model8, X_train, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)
  
train_results_O1 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_O1, col_O1[0]), 'train_results_O1_MLP')
save(train_results_O1,'train_results_O1_normalized_MLP')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_MLP(model8, X_val, window_val, y_val)
val_results_O1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_O1, col_O1[0]),'val_results_O1_MLP')
save(val_results_O1,'val_results_O1_normalized_MLP')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_MLP(model8, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_O1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_O1, col_O1[0]), 'test_results_O1_MLP')
save(test_results_O1,'test_results_O1_normalized_MLP')

In [ ]:
#LINEAR
#Model creation linear
model8 = LinearRegression()
#Model training linear
X_train = train_file_data_MLP[col_O1[1:] + ['y_k_1','y_k_2']]
y_train = train_file_data_MLP[col_O1[0]]

model8.fit(X_train, y_train)

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
train_predictions = predict_LINEAR(model8, X_train, window_train, y_train)
train_results_O1 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_O1, col_O1[0]),'train_results_O1_LINEAR')
save(train_results_O1,'train_results_O1_normalized_LINEAR')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_LINEAR(model8, X_val, window_val, y_val)
val_results_O1 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_O1, col_O1[0]),'val_results_O1_LINEAR')
save(val_results_O1,'val_results_O1_normalized_LINEAR')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_LINEAR(model8, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_O1 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_O1, col_O1[0]), 'test_results_O1_LINEAR')
save(test_results_O1,'test_results_O1_normalized_LINEAR')


In [ ]:
#Displaying results MLP
display_test_results(get('test_results_O1_MLP'), "MODEL 8 - Model pomiaru zawartości tlenu w spalinach O1 MLP ARX")

In [ ]:
display_train_val_test(get('train_results_O1_MLP'), get('val_results_O1_MLP'), get('test_results_O1_MLP'), "MODEL 8 MLP ARX - Model pomiaru zawartości tlenu w spalinach O1", y_unit='%')

In [ ]:
#Displaying results LINEAR
display_test_results(get('test_results_O1_LINEAR'), "MODEL 8 - Model pomiaru zawartości tlenu w spalinach O1 LINEAR ARX")

In [ ]:
display_train_val_test(get('train_results_O1_LINEAR'), get('val_results_O1_LINEAR'), get('test_results_O1_LINEAR'), "MODEL 8 LINEAR ARX - Model pomiaru zawartości tlenu w spalinach O1", y_unit='%')

## Metrics

In [ ]:
metrics = []
files = ["train_results_O1_normalized_LSTM", "val_results_O1_normalized_LSTM", "test_results_O1_normalized_LSTM", "train_results_O1_normalized_MLP", "val_results_O1_normalized_MLP", "test_results_O1_normalized_MLP", "train_results_O1_normalized_LINEAR", "val_results_O1_normalized_LINEAR", "test_results_O1_normalized_LINEAR"]
for file in files:
  results = get(file).drop(columns=["Unnamed: 0"])
  me, mse, mae, rmse = calculate_metrics(results[results.columns[1]], results[results.columns[0]])#Actuals and then predicitons
  metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df['Model'] = ['LSTM train', 'LSTM val', 'LSTM test', 'MLP ARX train', 'MLP ARX val', 'MLP ARX test', 'LINEAR ARX train', 'LINEAR ARX val', 'LINEAR ARX test']
metrics_df.rename(columns={'Model': 'MODEL 8 - Model pomiaru zawartości tlenu w spalinach O1', 'ME': 'Mean_Error', 'MSE': 'Mean_Squared_Error', 
                           'MAE': 'Mean_Absolute_Error', 'RMSE': 'Root_Mean_Squared_Error'}, inplace=True)
metrics_df.to_csv("metrics/O1_models_metrics_NEW.csv", index=False)

In [ ]:
metrics_df = pd.read_csv("metrics/O1_models_metrics_NEW.csv")
metrics_df.iloc[6:, [0, 2, 4]].style.apply(highlight_min_green, subset=metrics_df.iloc[:, [ 2, 4]].columns)#['Mean_Squared_Error', 'Root_Mean_Squared_Error'], axis=0)

# MODEL 9 - Model pomiaru zawartości tlenu w spalinach O2

In [ ]:
#Comparison
display_comparison(get('test_results_O2_LSTM'), get('test_results_O2_MLP'), get('test_results_O2_LINEAR'), "MODEL 9 - Model pomiaru zawartości tlenu w spalinach O2", y_unit = '%')

In [ ]:
metrics = []

## LSTM

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  df_O2 = file_data[col_O2]
  
  X_train, y_train = data_to_X_y(df_O2[col_O2[0]], WINDOW_SIZE)
  var_train = df_O2[col_O2[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data[col_O2[0]], WINDOW_SIZE)
var_val = val_file_data[col_O2[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

In [ ]:
#Model creation
model9 = create_model(6) #6 additional variables not including sequences

cp = ModelCheckpoint('model_O2/model_O2.keras', save_best_only=True)

model9.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model9.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model predictions
model9 = load_model('model_O2/model_O2.keras')

X_test, y_test = data_to_X_y(test_file_data[col_O2[0]], WINDOW_SIZE)
var_test = test_file_data[col_O2[1:]][WINDOW_SIZE-1:].to_numpy()
#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]

test = 0
window_test = X_test.shape[0]
# train = 0
# window_train = X_train.shape[0]
# val = 0
# window_val = X_val.shape[0]


# train_predictions = predict(model9, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_O2 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(denorm(train_results_O2, col_O2[0]),'train_results_O2')
# save(train_results_O2,'train_results_O2_normalized')

# val_predictions = predict(model9, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_O2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(denorm(val_results_O2, col_O2[0]),'val_results_O2')
# save(val_results_O2,'val_results_O2_normalized')

test_predictions = predict(model9, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test)
test_results_O2 = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_O2, col_O2[0]), 'test_results_O2')
save(test_results_O2,'test_results_O2_normalized')

In [ ]:
#Displaying results
display_test_results(get('test_results_O2'), "MODEL 9 - Model pomiaru zawartości tlenu w spalinach O2 LSTM")
#display(get('train_results_O2'), get('val_results_O2'), get('test_results_O2'),'all', "MODEL 9 - Model pomiaru zawartości tlenu w spalinach O2 LSTM")

## LSTM resampled data

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
   #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  df_O2 = file_data[col_O2]
  
  X_train, y_train = data_to_X_y(df_O2[col_O2[0]], WINDOW_SIZE)
  var_train = df_O2[col_O2[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data_resampled[col_O2[0]], WINDOW_SIZE)
var_val = val_file_data_resampled[col_O2[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_O2 = file_data[col_O2]
  
  X_test, y_test = data_to_X_y(df_O2[col_O2[0]], WINDOW_SIZE)
  var_test = df_O2[col_O2[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_test = min(X_test.shape[0],min(y_test.shape[0], var_test.shape[0]))
  X_test = X_test[:length_test] 
  y_test = y_test[:length_test] 
  var_test = var_test[:length_test] 
  
  test_data.append([X_test, y_test, var_test])

In [ ]:
#Model creation
model9 = create_model(6) #6 additional variables not including sequences

cp = ModelCheckpoint('model_O2/model_O2_LSTM.keras', save_best_only=True)

model9.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model9.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model9 = load_model('model_O2/model_O2_LSTM.keras')
train_results = []
for [X_train, y_train, var_train] in prepared_data:
  #We specify from which point we want to predict the next values for each dataset [s]
  #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict(model9, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)

train_results_O2 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_O2, col_O2[0]), 'train_results_O2_LSTM')
save(train_results_O2,'train_results_O2_normalized_LSTM')

val = 0
window_val = X_val.shape[0]
val_predictions = predict(model9, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val, y_val)
val_results_O2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_O2, col_O2[0]),'val_results_O2_LSTM')
save(val_results_O2,'val_results_O2_normalized_LSTM')

test_results = []
for [X_test, y_test, var_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict(model9, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)

test_results_O2 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_O2, col_O2[0]), 'test_results_O2_LSTM')
save(test_results_O2,'test_results_O2_normalized_LSTM')

In [ ]:
#Displaying results
display_test_results(get('test_results_O2_LSTM'), "MODEL 9 - Model pomiaru zawartości tlenu w spalinach O2 LSTM")

In [ ]:
display_train_val_test(get('train_results_O2_LSTM'), get('val_results_O2_LSTM'), get('test_results_O2_LSTM'), "MODEL 9 LSTM - Model pomiaru zawartości tlenu w spalinach O2", y_unit='%')

## MLP ARX and LINEAR ARX resampled data

In [ ]:
#Data preparation
#Przetrwarzamy dane tak, żeby można było przekazać dwa poprzednie kroki zmiennej jako parametry do naszego modelu

prepared_data = []
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  #Taking only the columns relevant for the model
  df_O2 = file_data[col_O2]
  
  # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_O2['y_k_1'] = df_O2[col_O2[0]].shift(1)  # y[k-1]
  df_O2['y_k_2'] = df_O2[col_O2[0]].shift(2)  # y[k-2]
  df_O2 = df_O2.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_train = df_O2[col_O2[1:] + ['y_k_1','y_k_2']]
  y_train = df_O2[col_O2[0]]
  
  prepared_data.append([X_train, y_train])
  
X_val = val_file_data_MLP[col_O2[1:] + ['y_k_1','y_k_2']]
y_val = val_file_data_MLP[col_O2[0]]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_O2 = file_data[col_O2]
  
    # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_O2['y_k_1'] = df_O2[col_O2[0]].shift(1)  # y[k-1]
  df_O2['y_k_2'] = df_O2[col_O2[0]].shift(2)  # y[k-2]
  df_O2 = df_O2.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_test = df_O2[col_O2[1:] + ['y_k_1','y_k_2']]
  y_test = df_O2[col_O2[0]]
  
  test_data.append([X_test, y_test])


In [ ]:
#Model creation
model9 = create_model_MLP(8) #8 variables (6+2y) 

cp = ModelCheckpoint('model_O2/model_O2_MLP.keras', save_best_only=True)

model9.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model9.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model9 = load_model('model_O2/model_O2_MLP.keras')

train_results = []
for [X_train, y_train] in prepared_data:
  # #We specify from which point we want to predict the next values for each dataset [s]
  # #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict_MLP(model9, X_train, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)
  
train_results_O2 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_O2, col_O2[0]), 'train_results_O2_MLP')
save(train_results_O2,'train_results_O2_normalized_MLP')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_MLP(model9, X_val, window_val, y_val)
val_results_O2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_O2, col_O2[0]),'val_results_O2_MLP')
save(val_results_O2,'val_results_O2_normalized_MLP')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_MLP(model9, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_O2 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_O2, col_O2[0]), 'test_results_O2_MLP')
save(test_results_O2,'test_results_O2_normalized_MLP')

In [ ]:
#LINEAR
#Model creation linear
model9 = LinearRegression()
#Model training linear
X_train = train_file_data_MLP[col_O2[1:] + ['y_k_1','y_k_2']]
y_train = train_file_data_MLP[col_O2[0]]

model9.fit(X_train, y_train)

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
train_predictions = predict_LINEAR(model9, X_train, window_train, y_train)
train_results_O2 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_O2, col_O2[0]),'train_results_O2_LINEAR')
save(train_results_O2,'train_results_O2_normalized_LINEAR')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_LINEAR(model9, X_val, window_val, y_val)
val_results_O2 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_O2, col_O2[0]),'val_results_O2_LINEAR')
save(val_results_O2,'val_results_O2_normalized_LINEAR')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_LINEAR(model9, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_O2 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_O2, col_O2[0]), 'test_results_O2_LINEAR')
save(test_results_O2,'test_results_O2_normalized_LINEAR')


In [ ]:
#Displaying results MLP
display_test_results(get('test_results_O2_MLP'), "MODEL 9 - Model pomiaru zawartości tlenu w spalinach O2 MLP ARX")

In [ ]:
display_train_val_test(get('train_results_O2_MLP'), get('val_results_O2_MLP'), get('test_results_O2_MLP'), "MODEL 9 MLP ARX - Model pomiaru zawartości tlenu w spalinach O2", y_unit='%')

In [ ]:
#Displaying results LINEAR
display_test_results(get('test_results_O2_LINEAR'), "MODEL 9 - Model pomiaru zawartości tlenu w spalinach O2 LINEAR ARX")

In [ ]:
display_train_val_test(get('train_results_O2_LINEAR'), get('val_results_O2_LINEAR'), get('test_results_O2_LINEAR'), "MODEL 9 LINEAR ARX - Model pomiaru zawartości tlenu w spalinach O2", y_unit='%')

## Metrics

In [ ]:
metrics = []
files = ["train_results_O2_normalized_LSTM", "val_results_O2_normalized_LSTM", "test_results_O2_normalized_LSTM", "train_results_O2_normalized_MLP", "val_results_O2_normalized_MLP", "test_results_O2_normalized_MLP", "train_results_O2_normalized_LINEAR", "val_results_O2_normalized_LINEAR", "test_results_O2_normalized_LINEAR"]
for file in files:
  results = get(file).drop(columns=["Unnamed: 0"])
  me, mse, mae, rmse = calculate_metrics(results[results.columns[1]], results[results.columns[0]])#Actuals and then predicitons
  metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df['Model'] = ['LSTM train', 'LSTM val', 'LSTM test', 'MLP ARX train', 'MLP ARX val', 'MLP ARX test', 'LINEAR ARX train', 'LINEAR ARX val', 'LINEAR ARX test']
metrics_df.rename(columns={'Model': 'MODEL 9 - Model pomiaru zawartości tlenu w spalinach O2', 'ME': 'Mean_Error', 'MSE': 'Mean_Squared_Error', 
                           'MAE': 'Mean_Absolute_Error', 'RMSE': 'Root_Mean_Squared_Error'}, inplace=True)
metrics_df.to_csv("metrics/O2_models_metrics_NEW.csv", index=False)

In [ ]:
metrics_df = pd.read_csv("metrics/O2_models_metrics_NEW.csv")
metrics_df.iloc[6:, [0, 2, 4]].style.apply(highlight_min_green, subset=metrics_df.iloc[:, [ 2, 4]].columns)#['Mean_Squared_Error', 'Root_Mean_Squared_Error'], axis=0)

# MODEL 10 - Model pomiaru zawartości tlenu w spalinach O3

In [ ]:
#Comparison
display_comparison(get('test_results_O3_LSTM'), get('test_results_O3_MLP'), get('test_results_O3_LINEAR'), "MODEL 10 - Model pomiaru zawartości tlenu w spalinach O3", y_unit = '%')

In [ ]:
metrics = []

## LSTM

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  df_O3 = file_data[col_O3]
  
  X_train, y_train = data_to_X_y(df_O3[col_O3[0]], WINDOW_SIZE)
  var_train = df_O3[col_O3[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data[col_O3[0]], WINDOW_SIZE)
var_val = val_file_data[col_O3[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

In [ ]:
#Model creation
model10 = create_model(12) #12 additional variables not including sequences

cp = ModelCheckpoint('model_O3/model_O3.keras', save_best_only=True)

model10.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model10.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model predictions
model10 = load_model('model_O3/model_O3.keras')

X_test, y_test = data_to_X_y(test_file_data[col_O3[0]], WINDOW_SIZE)
var_test = test_file_data[col_O3[1:]][WINDOW_SIZE-1:].to_numpy()
#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]

test = 0
window_test = X_test.shape[0]
# train = 0
# window_train = X_train.shape[0]
# val = 0
# window_val = X_val.shape[0]


# train_predictions = predict(model10, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train)
# train_results_O3 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
# save(denorm(train_results_O3, col_O3[0]),'train_results_O3')
# save(train_results_O3,'train_results_O3_normalized')

# val_predictions = predict(model10, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val)
# val_results_O3 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
# save(denorm(val_results_O3, col_O3[0]),'val_results_O3')
# save(val_results_O3,'val_results_O3_normalized')

test_predictions = predict(model10, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test)
test_results_O3 = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
save(denorm(test_results_O3, col_O3[0]), 'test_results_O3')
save(test_results_O3,'test_results_O3_normalized')

In [ ]:
#Displaying results
display_test_results(get('test_results_O3'), "MODEL 10 - Model pomiaru zawartości tlenu w spalinach O3 LSTM")
#display(get('train_results_O3'), get('val_results_O3'), get('test_results_O3'),'all', "MODEL 10 - Model pomiaru zawartości tlenu w spalinach O3 LSTM")

## LSTM resampled data

In [ ]:
#Data preparation
prepared_data = []
WINDOW_SIZE = 10
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
   #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  df_O3 = file_data[col_O3]
  
  X_train, y_train = data_to_X_y(df_O3[col_O3[0]], WINDOW_SIZE)
  var_train = df_O3[col_O3[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_train = min(X_train.shape[0],min(y_train.shape[0], var_train.shape[0]))
  X_train = X_train[:length_train] 
  y_train = y_train[:length_train]
  var_train = var_train[:length_train] 
  
  prepared_data.append([X_train, y_train, var_train])

X_val, y_val = data_to_X_y(val_file_data_resampled[col_O3[0]], WINDOW_SIZE)
var_val = val_file_data_resampled[col_O3[1:]][WINDOW_SIZE-1:].to_numpy()

length_val = min(X_val.shape[0],min(y_val.shape[0], var_val.shape[0]))
X_val = X_val[:length_val] 
y_val = y_val[:length_val] 
var_val = var_val[:length_val]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_O3 = file_data[col_O3]
  
  X_test, y_test = data_to_X_y(df_O3[col_O3[0]], WINDOW_SIZE)
  var_test = df_O3[col_O3[1:]][WINDOW_SIZE-1:].to_numpy()
  
  length_test = min(X_test.shape[0],min(y_test.shape[0], var_test.shape[0]))
  X_test = X_test[:length_test] 
  y_test = y_test[:length_test] 
  var_test = var_test[:length_test] 
  
  test_data.append([X_test, y_test, var_test])

In [ ]:
#Model creation
model10 = create_model(12) #12 additional variables not including sequences

cp = ModelCheckpoint('model_O3/model_O3_LSTM.keras', save_best_only=True)

model10.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train, var_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model10.fit([X_train, var_train], y_train, validation_data=([X_val, var_val], y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model10 = load_model('model_O3/model_O3_LSTM.keras')
train_results = []
for [X_train, y_train, var_train] in prepared_data:
  #We specify from which point we want to predict the next values for each dataset [s]
  #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict(model10, X_train, var_train[train:train+window_train], WINDOW_SIZE, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)

train_results_O3 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_O3, col_O3[0]), 'train_results_O3_LSTM')
save(train_results_O3,'train_results_O3_normalized_LSTM')

val = 0
window_val = X_val.shape[0]
val_predictions = predict(model10, X_val, var_val[val:val+window_val], WINDOW_SIZE, window_val, y_val)
val_results_O3 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_O3, col_O3[0]),'val_results_O3_LSTM')
save(val_results_O3,'val_results_O3_normalized_LSTM')

test_results = []
for [X_test, y_test, var_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict(model10, X_test, var_test[test:test+window_test], WINDOW_SIZE, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)

test_results_O3 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_O3, col_O3[0]), 'test_results_O3_LSTM')
save(test_results_O3,'test_results_O3_normalized_LSTM')

In [ ]:
#Displaying results
display_test_results(get('test_results_O3_LSTM'), "MODEL 10 - Model pomiaru zawartości tlenu w spalinach O3 LSTM")

In [ ]:
display_train_val_test(get('train_results_O3_LSTM'), get('val_results_O3_LSTM'), get('test_results_O3_LSTM'), "MODEL 10 LSTM - Model pomiaru zawartości tlenu w spalinach O3", y_unit='%')

## MLP ARX and LINEAR ARX resampled data

In [ ]:
#Data preparation
#Przetrwarzamy dane tak, żeby można było przekazać dwa poprzednie kroki zmiennej jako parametry do naszego modelu

prepared_data = []
for file in train_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True) 
  
  #Taking only the columns relevant for the model
  df_O3 = file_data[col_O3]
  
  # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_O3['y_k_1'] = df_O3[col_O3[0]].shift(1)  # y[k-1]
  df_O3['y_k_2'] = df_O3[col_O3[0]].shift(2)  # y[k-2]
  df_O3 = df_O3.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_train = df_O3[col_O3[1:] + ['y_k_1','y_k_2']]
  y_train = df_O3[col_O3[0]]
  
  prepared_data.append([X_train, y_train])
  
X_val = val_file_data_MLP[col_O3[1:] + ['y_k_1','y_k_2']]
y_val = val_file_data_MLP[col_O3[0]]

test_data = []
for file in test_file_paths:
  file_data = norm(pd.read_csv(file, delimiter=';')) #normalizujemy dane z przekazanego pliku
  file_data = file_data.iloc[2:].reset_index(drop=True) #usuwamy dwa pierwsze wiersze i resetujemy index
  
  #Resampling the data -> taking every 10th row
  file_data = file_data.iloc[::10]
  file_data.reset_index(inplace= True, drop= True)
  
  df_O3 = file_data[col_O3]
  
    # Creating lagged versions of the target variable (y[k-1], y[k-2])
  df_O3['y_k_1'] = df_O3[col_O3[0]].shift(1)  # y[k-1]
  df_O3['y_k_2'] = df_O3[col_O3[0]].shift(2)  # y[k-2]
  df_O3 = df_O3.dropna().reset_index(drop=True) #Removing the first two rows where y[k-2] or y[k-1] would be NaN
  
  X_test = df_O3[col_O3[1:] + ['y_k_1','y_k_2']]
  y_test = df_O3[col_O3[0]]
  
  test_data.append([X_test, y_test])


In [ ]:
#Model creation
model10 = create_model_MLP(14) #14 variables (12+2y)

cp = ModelCheckpoint('model_O3/model_O3_MLP.keras', save_best_only=True)

model10.compile(loss = MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
#Model training
count = 1
for [X_train, y_train] in prepared_data:
  print("\n", "File ",  count, "/", len(train_file_paths), "\n")
  model10.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, callbacks = [cp])
  count=count+1

In [ ]:
#Model making predictions
model10 = load_model('model_O3/model_O3_MLP.keras')

train_results = []
for [X_train, y_train] in prepared_data:
  # #We specify from which point we want to predict the next values for each dataset [s]
  # #We specify the predict horizon from the starting point [s]
  train = 0
  window_train = X_train.shape[0]
  train_predictions = predict_MLP(model10, X_train, window_train, y_train)
  partial_results = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
  train_results.append(partial_results)
  
train_results_O3 = pd.concat(train_results, ignore_index=True)
save(denorm(train_results_O3, col_O3[0]), 'train_results_O3_MLP')
save(train_results_O3,'train_results_O3_normalized_MLP')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_MLP(model10, X_val, window_val, y_val)
val_results_O3 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_O3, col_O3[0]),'val_results_O3_MLP')
save(val_results_O3,'val_results_O3_normalized_MLP')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_MLP(model10, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_O3 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_O3, col_O3[0]), 'test_results_O3_MLP')
save(test_results_O3,'test_results_O3_normalized_MLP')

In [ ]:
#LINEAR
#Model creation linear
model10 = LinearRegression()
#Model training linear
X_train = train_file_data_MLP[col_O3[1:] + ['y_k_1','y_k_2']]
y_train = train_file_data_MLP[col_O3[0]]

model10.fit(X_train, y_train)

#We specify from which point we want to predict the next values for each dataset [s]
#We specify the predict horizon from the starting point [s]
train = 0
window_train = X_train.shape[0]
train_predictions = predict_LINEAR(model10, X_train, window_train, y_train)
train_results_O3 = pd.DataFrame(data = {'Train Predictions':np.squeeze(train_predictions), 'Actuals':y_train[train:train+window_train]})
save(denorm(train_results_O3, col_O3[0]),'train_results_O3_LINEAR')
save(train_results_O3,'train_results_O3_normalized_LINEAR')

val = 0
window_val = X_val.shape[0]
val_predictions = predict_LINEAR(model10, X_val, window_val, y_val)
val_results_O3 = pd.DataFrame(data = {'Val Predictions':np.squeeze(val_predictions), 'Actuals':y_val[val:val+window_val]})
save(denorm(val_results_O3, col_O3[0]),'val_results_O3_LINEAR')
save(val_results_O3,'val_results_O3_normalized_LINEAR')

test_results = []
for [X_test, y_test] in test_data:
  test = 0
  window_test = X_test.shape[0]
  test_predictions = predict_LINEAR(model10, X_test, window_test, y_test)
  partial_results = pd.DataFrame(data = {'Test Predictions':np.squeeze(test_predictions), 'Actuals':y_test[test:test+window_test]})
  test_results.append(partial_results)
  
test_results_O3 = pd.concat(test_results, ignore_index=True)
save(denorm(test_results_O3, col_O3[0]), 'test_results_O3_LINEAR')
save(test_results_O3,'test_results_O3_normalized_LINEAR')


In [ ]:
#Displaying results MLP
display_test_results(get('test_results_O3_MLP'), "MODEL 10 - Model pomiaru zawartości tlenu w spalinach O3 MLP ARX")

In [ ]:
display_train_val_test(get('train_results_O3_MLP'), get('val_results_O3_MLP'), get('test_results_O3_MLP'), "MODEL 10 MLP ARX - Model pomiaru zawartości tlenu w spalinach O3", y_unit='%')

In [ ]:
#Displaying results LINEAR
display_test_results(get('test_results_O3_LINEAR'), "MODEL 10 - Model pomiaru zawartości tlenu w spalinach O3 LINEAR ARX")

In [ ]:
display_train_val_test(get('train_results_O3_LINEAR'), get('val_results_O3_LINEAR'), get('test_results_O3_LINEAR'), "MODEL 10 LINEAR ARX- Model pomiaru zawartości tlenu w spalinach O3", y_unit='%')

## Metrics

In [ ]:
metrics = []
files = ["train_results_O3_normalized_LSTM", "val_results_O3_normalized_LSTM", "test_results_O3_normalized_LSTM", "train_results_O3_normalized_MLP", "val_results_O3_normalized_MLP", "test_results_O3_normalized_MLP", "train_results_O3_normalized_LINEAR", "val_results_O3_normalized_LINEAR", "test_results_O3_normalized_LINEAR"]
for file in files:
  results = get(file).drop(columns=["Unnamed: 0"])
  me, mse, mae, rmse = calculate_metrics(results[results.columns[1]], results[results.columns[0]])#Actuals and then predicitons
  metrics.append({'Model': 'model', 'ME': me, 'MSE': mse, 'MAE': mae, 'RMSE': rmse})

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df['Model'] = ['LSTM train', 'LSTM val', 'LSTM test', 'MLP ARX train', 'MLP ARX val', 'MLP ARX test', 'LINEAR ARX train', 'LINEAR ARX val', 'LINEAR ARX test']
metrics_df.rename(columns={'Model': 'MODEL 10 - Model pomiaru zawartości tlenu w spalinach O3', 'ME': 'Mean_Error', 'MSE': 'Mean_Squared_Error', 
                           'MAE': 'Mean_Absolute_Error', 'RMSE': 'Root_Mean_Squared_Error'}, inplace=True)
metrics_df.to_csv("metrics/O3_models_metrics_NEW.csv", index=False)

In [ ]:
metrics_df = pd.read_csv("metrics/O3_models_metrics_NEW.csv")
metrics_df.iloc[6:, [0, 2, 4]].style.apply(highlight_min_green, subset=metrics_df.iloc[:, [ 2, 4]].columns)#['Mean_Squared_Error', 'Root_Mean_Squared_Error'], axis=0)